<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 13 : Gestion des processus et des ressources</h1>

Considérons une activité simple sur ordinateur : Alice rédige un compte rendu pour un projet informatique.  
* Elle a *ouvert* un logiciel de traitement de texte pour écrire le rapport.  
* Son navigateur web est aussi ouvert avec divers onglets, l'un pointant vers Wikipedia, l'autre vers un moteur de recherche et un troisième vers un site de réseau social dont elle se sert pour partager son humeur avec ses camarades.  
* Elle utilise un logiciel de dessin afin d'ajouter des illustrations à son compte-rendu.  
* Son projet étant en Python, elle dispose aussi d'une fenêtre avec l'environnement Idle dans lequel elle exécute son programme afin d'en vérifier les résultats.  
* Enfin, pour ne pas être perturbée, elle a mis des écouteurs sur ses oreilles et écoute de la musique grâce au lecteur de musique de son ordinateur.

Tous ces programmes s'exécutent *en même temps*.  
Pourtant, si on se souvient de la façon dont sont construits les ordinateurs, ils ne disposent que d'un nombre limité de processeurs (la norme est de un à huit processeurs (on parle de *cœurs*, core en anglais) pour les ordinateurs personnels, téléphones et tablettes, de l'ordre de quelques dizaines de processeurs pour les machines de type serveur, et plusieurs milliers pour les super calculateurs).  
Or, comme on le sait, un programme n'est qu'une suite d'instructions en langage machine, ces dernières étant exécutées une à une par le processeur.  
Comment le processeur peut-il donc exécuter *en même temps* les instructions du programme de traitement de texte et celles du lecteur de musique?

Cette exécution **concurrente** de programmes est l'une des fonctionnalités de base offertes par les systèmes d'exploitation modernes.  
On parle alors de systèmes d'exploitation multitâches.  
Nous rappelons d'abord brièvement comment un programme est exécuté par le système d'exploitation, puis nous introduisons le principe de fonctionnement de l'**ordonnanceur de processus**,
la partie du système d'exploitation permettant l'exécution concurrente des programmes.

## L'ordonnanceur
### Rappel sur l'exécution d'un programme
Un exécutable est un **fichier** (par exemple stocké sur le disque dur) contenant une suite d'instructions en langage machine (on appelle aussi souvent ces fichiers, des **binaires**, de l'anglais *binary*, abréviation de *binary code executable file*, fichier de code binaire exécutable).  
C'est donc une suite d'octets que le processeur est capable de décoder et exécuter.  
Concrètement, lorsque l'on exécute un programme (par exemple, en cliquant sur l'icône du fichier exécutable ou en renseignant son chemin dans un terminal), le système d'exploitation effectue les actions suivantes :
1. le fichier contenant le programme (l'exécutable) est copié dans la mémoire RAM, à une certaine adresse `a`
2. le système d'exploitation écrit la valeur `a` dans le registre PC (Program Counter, parfois appelé Instruction Pointer)

Au prochain cycle d'horloge du processeur, ce dernier va, alors, lire l'instruction se trouvant à l'adresse `a` et l'exécuter.  
Une fois cela fait, il exécutera ensuite la seconde instruction et ainsi de suite...

On rappelle que l'exécution d'une instruction se décompose, elle-même, en plusieurs sous-étapes effectuées au sein du processeur : 
* le chargement (récupérer l'instruction en mémoire)
* le décodage (déterminer, dans la suite d'octets chargés, quelle instruction ils encodent) 
* l'exécution proprement dite

<div style="text-align: center">
   <img alt="Cycle CPU" src="Images/CPU_FDX.svg" > 
</div>


### Interruptions
Même si elle est correcte, la description que nous avons faite de l'exécution d'un programme est incomplète.  
En effet, si rien de plus n'est fait, alors la seule chose que l'on peut attendre, c'est que le programme en question s'exécute jusqu'à sa dernière instruction, puis rende la main au système d'exploitation.  
Impossible alors de l'interrompre!  
Impossible aussi de pouvoir exécuter deux programmes en même temps.  
Pour pallier ce problème, les systèmes d'exploitation utilisent une fonctionnalité importante des processeurs modernes : la notion d'**interruption**.

Une interruption est un **signal** envoyé au processeur lorsqu'un événement se produit.  
Il existe plusieurs types d'interruptions.  

Certaines sont générées par le matériel (par exemple, un disque dur signale qu'il a fini d'écrire des octets, une carte réseau signale que des paquets de données arrivent, etc.).  
Lorsque le processeur reçoit une interruption, il **interrompt son exécution** à la fin de l'instruction courante et exécute un programme se trouvant à une adresse prédéfinie. Ce programme reçoit en argument une copie des valeurs courantes des registres, ainsi qu'un code numérique lui permettant de savoir à quel type d'interruption il fait face.  
Ce programme spécial s'appelle le **gestionnaire d'interruption**. Il est installé à une certaine adresse mémoire par le système d'exploitation, très tôt après le démarrage de la machine.  

La réalité est un peu plus complexe : une unité spéciale du processeur, le contrôleur d'interruptions programmable ou PIC (pour l'anglais *Programmable Interrupt Controler*) dispose d'instructions assembleur dédiées permettant la configuration et la gestion des interruptions.

Parmi les interruptions matérielles, on retrouve les **interruptions d'horloge**. Le processeur génère de lui-même une interruption matérielle à intervalles de temps fixe.  
Historiquement, sur processeur Intel, cette interruption était levée toutes les 55 ms (environ 18 fois par seconde). Le gestionnaire d'interruption était donc appelé au moins toutes les 55 ms.  
De nos jours, les processeurs disposent d'horloges de haute précision capables d'émettre des interruptions avec une fréquence de 10 Mhz, donc toutes les 100 ns.  
Ces interruptions d'horloges, alliées au gestionnaire d'interruption, sont les pièces essentielles permettant d'exécuter des programmes de façon concurrente.

### Vocabulaire
Certains termes sont particulièrement importants pour la suite et doivent être définis précisément :
* **Exécutable** : un fichier binaire contenant des instructions machines directement exécutables par le processeur de la machine.

* **Processus** (*programme en cours d'exécution*) : un processus est le phénomène dynamique qui correspond à l'exécution d'un programme particulier.  
Le système d'exploitation identifie, généralement, les processus par un numéro unique.  
Un processus est décrit par :
* l'ensemble de la mémoire allouée par le système pour l'exécution de ce programme (ce qui inclut le code exécutable copié en mémoire et toutes les données manipulées par le programme, sur la pile ou dans le tas)
* l'ensemble des ressources utilisées par le programme (fichiers ouverts, connexions réseaux, etc.)
* les valeurs stockées dans tous les registres du processeur.

* **Thread** (ou tâche) : exécution d'une suite d'instructions démarrée par un processus.  
Deux processus sont l'exécution de deux programmes (par exemple, un traitement de texte et un navigateur web).  
Deux threads sont l'exécution concurrente de deux suites d'instructions d'un même processus.  
Par exemple, pour un navigateur web, il peut y avoir un thread dont le rôle est de dessiner la page web dans une fenêtre et un autre thread dont le rôle est de télécharger un fichier sur lequel l'utilisateur a cliqué.  
La différence fondamentale entre processus et thread est que les processus ne partagent pas leur mémoire, alors que les threads, issus d'un même processus, peuvent accéder aux variables globales du programme et occupent le même espace en mémoire.

* **Exécution concurrente** : deux processus ou tâches s'exécutent de manière concurrente si les intervalles de temps entre le début et la fin de leur exécution ont une partie commune.

* **Exécution parallèle** : deux processus ou tâches s'exécutent en parallèle s'ils s'exécutent au même instant.  
Pour que deux processus s'éxécutent en parallèle, il faut donc plusieurs processeurs sur la machine.  
C'est une légère simplification : les processeurs modernes, même mono-cœur, disposent de plusieurs unités arithmétiques et logiques, ce qui leur permet d'exécuter certains opérations en parallèle. Par exemple, ils peuvent exécuter deux additions simultanément.

### Ordonnanceur du système d'exploitation
Comme nous l'avons vu, le système d'exploitation peut configurer une horloge et le gestionnaire d'interruption pour *reprendre la main*, c'est-à-dire, exécuter du code qui lui est propre, à intervalles réguliers.  
Lorsqu'il s'exécute, il peut, entre autres choses, décider **à quel programme en cours d'exécution** il va rendre la main.  

Dans le scénario d'utilisation donné en introduction, l'ordonnanceur fonctionnera donc de la façon suivante :

1 . Le programme *traitement de texte* est en cours d'exécution (l'utilisateur saisit du texte qui est affiché à l'écran, sauvegarde dans un fichier, etc.).

2 . Une interruption d'horloge se déclenche.

3 . Le code du gestionnaire d'interruption est appelé.  
Il reçoit, en argument, les valeurs qu'ont tous les registres avant le déclenchement de l'interruption (donc tout l'état *interne* du traitement de texte).

4 . Le gestionnaire d'interruption sauvegarde ces registres à un endroit particulier de la mémoire.

5 . Il choisit, dans la liste des processus, un autre processus, par exemple celui correspondant au navigateur web.

6 . Il restaure les valeurs de tous les registres du processeur qu'il a sauvegardées la dernière fois qu'il a interrompu le navigateur web.  
Parmi ces registres sauvegardés, il y a notamment IP, l'adresse de la prochaine instruction à exécuter.  
Elle pointait alors vers une instruction du programme *navigateur web*.

7 . Le gestionnaire d'interruption rend la main.  
La prochaine instruction à exécuter est celle du processus navigateur web, qui reprend son exécution, jusqu'à ce qu'il soit mis en pause par la prochaine interruption d'horloge.

Le fait que l'ordonnanceur interrompe un processus et sauve son état s'appelle une **commutation de contexte**.  
Afin de pouvoir choisir, parmi tous les processus, lequel exécuter lors de la prochaine interruption, le système d'exploitation conserve, pour chaque processus, une structure de données nommée PCB (pour l'anglais *Process Control Bloc* ou **bloc de contrôle du processus**).  
Le PCB est simplement une zone mémoire dans laquelle sont stockées diverses informations sur le processus.

| Nom        | Description                                                                    |
|:------------|:--------------------------------------------------------------------------------|
| PID        | Process ID, l'identifiant numérique du processus                               |
| État       | l'état dans lequel se trouve le processus                                      |
| Registres  | la valeur des registres lors de sa dernière interruption                       |
| Mémoire    | zone mémoire (plage d'adresses) allouée par le processus lors de son exécution |
| Ressources | liste des fichiers ouverts, connexions réseaux en cours d'utilisation, etc.    |

Pour choisir, parmi les processus, celui auquel il va donner la main, l'ordonnanceur conserve les PCB dans une structure de donnée (par exemple, une file).  
Le premier processus dans la file reprend son exécution. Lors de la prochaine interruption, il est mis en bout de file.  
Cette stratégie simple permet d'éviter qu'un processus monopolise tout le temps de calcul.  
En effet, avant qu'un processus mis en bout de file puisse s'exécuter de nouveau, il aura laissé une chance à tous les autres d'avoir la main.

### États des processus
Les interruptions d'horloges ne sont pas les seuls événements permettant d'interrompre les processus.  
Considérons le petit programme Python, ci-dessous :

In [ ]:
texte = input("Saisir une phrase : ")
print(f"Votre phrase en majuscules : {texte.upper()}")

Ce programme utilise la fonction `input` pour demander à l'utilisateur de saisir une phrase.  
Tant que l'utilisateur ne saisit rien, le programme ne peut pas avancer à l'instruction suivante.  
Le système d'exploitation (qui gère aussi le matériel et, en particulier, l'accès au clavier) peut suspendre le processus et le mettre en attente.  
De plus, il sait aussi qu'il est inutile de réveiller ce processus tant que l'utilisateur n'a pas interagi avec le clavier.  
Une telle interaction avec le clavier est signalée au processeur par une interruption, similaire aux interruptions d'horloges.  
De manière générale, lorsque des périphériques (carte réseau, disque dur, souris, clavier, etc.) veulent signaler au processeur qu'un événement est survenu, ils le feront au moyen d'une interruption.  
Le système d'exploitation aura alors l'occasion *de réveiller* l'un des processus parmi ceux qui attendaient un tel événement.

On voit donc que les processus peuvent être dans différents états.  
La plupart des systèmes d'exploitation utilisent principalement les états suivants :
* **Nouveau** : état d'un processus en cours de création.  
Le système d'exploitation vient de copier l'exécutable en mémoire et d'initialiser le PCB.

* **Prêt** : le processus peut être le prochain à s'exécuter.  
Il est dans la file des processus qui *attendent* leur tour et peuvent être choisis par l'ordonnanceur.

* **En exécution** : le processus est en train de s'exécuter.

* **En attente** : le processus est interrompu et en attente d'un événement externe (entrée/sortie, allocation mémoire, etc.).

* **Terminé** : le processus s'est terminé, le système d'exploitation est en train de désallouer les ressources que le processus utilisait.

Les états **Nouveau** et **Terminé** sont éphémères.  
En temps normal, l'état d'un processus variera entre **prêt**, **en attente** et **en exécution**.  

La figure suivante résume le cycle de vie d'un processus :

<div style="text-align: center">
   <img alt="Cycle de vie d'un processus" src="Images/processus-1.png" > 
</div>


On peut noter que, quel que soit l'état dans lequel se trouve un processus, il peut se terminer de façon anormale.  
* S'il est **en exécution**, cela peut être dû à une erreur provoquée par le programme (lecture d'une adresse mémoire invalide, division par 0, etc.). 
* Si le processus est **en attente** d'une entrée-sortie, il est possible qu'il se produise une erreur matérielle (disque dur défectueux, par exemple). 
* Enfin, si le processus est dans l'état **prêt**, il peut quand même se terminer de façon anormale si l'utilisateur qui a lancé le programme ou l'administrateur système décide de l'interrompre manuellement.

## Commandes Unix de gestion des processus
Dans les systèmes POSIX, la commande [`ps`](https://pubs.opengroup.org/onlinepubs/9699919799/utilities/ps.html) (pour l'anglais *process status* ou état des processus) permet d'obtenir des informations sur les processus en cours d'exécution.

```
utilisateur@machine:~$ ps -a -u -x
```
Les options `-a`, `-u` et `-x` permettent respectivement d'afficher tous les processus (et pas seulement ceux de l'utilisateur qui lance la commande), d'afficher le nom des utilisateurs (plutôt que leur identifiant numérique) et de compter aussi les processus n'ayant pas été lancés depuis un terminal (comme les *daemon* ou les processus lancés depuis une interface graphique).  
La commande affiche sur la sortie standard des informations sur les processus, comme, par exemple :


```
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.1 167880  7344 ?        Ss   09:26   2:40 /sbin/init sp
...
john        1438  0.0  0.0  11548  4952 tty2     Ss   15:45   0:00 bash
john        3537  5.4  3.6 998564 60764 ?        SI   15:12   9:11 /usr/bin/firefox
root        6524  0.0  0.0  29260  7780 ?        Ss   00:00   0:00 /usr/sbin/cupsd
john        6966  9.8  2.0 140692 24240 ?        SLI  15:41   2:56 /usr/bin/emacs
john        7490  0.0  0.0  11668  2704 tty2     R+   15:47   0:00 ps -a -u -x
...

```

Nous ne détaillons que les colonnes les plus importantes et, comme pour toute commande Unix, nous renvoyons le lecteur intéressé à la page de manuel de la commande accessible par la commande :

```
utilisateur@machine:~$ man ps
```

qui renvoie le manuel de la commance `ps` :

```sh
PS(1)                            User Commands                           PS(1)

NAME
       ps - report a snapshot of the current processes.

SYNOPSIS
       ps [options]

DESCRIPTION
       ps displays information about a selection of the active processes.  If
       you want a repetitive update of the selection and the displayed
       information, use top(1) instead.
       
...

```

* La colonne `USER` indique le nom de l'utilisateur qui a lancé le processus. 
* La colonne `PID` donne l'identifiant numérique du processus. 
* Les colonnes `%CPU` et `%MEM` indiquent respectivement le taux d'occupation du processeur et de la mémoire par le processus.  
Par exemple, dans l'affichage précédent, on peut voir que le processus `6966` occupe $9,8\%$ du temps de calcul du processeur et $2\%$ de la mémoire. En simplifiant un peu, on peut dire que sur les dernières $100$ secondes d'utilisation du système, $9,8$ secondes ont été passées à exécuter des instruction du processus `6966`. 
* La colonne `TTY` indique l'identifiant du terminal où le processus a été lancé.  
Un caractère `?` indique que le processus n'a pas été lancé depuis un terminal. 

* La colonne `STAT` indique l'état du processus (la première lettre en majuscule).  
Sur la plupart des systèmes Unix, les états sont:
* `R` : *running* ou *runnable*, le processus est dans l'état prêt ou en exécution (la commande `ps` ne différencie pas ces deux états)
* `S` : *sleeping*, le processus est **en attente**.

* Les colonnes `START` et `TIME` indiquent respectivement l'heure ou la date à laquelle le programme a été lancé et le temps cumulé d'exécution du processus correspondant (c'est-à-dire, le temps total pendant lequel le processus était dans l'état *en exécution*). 

* Enfin, la colonne `COMMAND` indique la ligne de commande utilisée pour lancer le programme (elle est tronquée dans notre exemple pour des raisons de place).

Une commande un peu plus conviviale et dynamique est la commande [`top`](https://man7.org/linux/man-pages/man1/top.1.html).  
Cette dernière affiche, en temps réel, des informations similaires à celles affichées par `ps`. Ces informations sont rafraîchies toutes les secondes.  
Cette commande est particulièrement précieuse pour essayer de déterminer quels processus occupent le plus le processeur ou ont alloué le plus de mémoire (sous le système Microsoft Windows, le gestionnaire de tâches joue un rôle similaire : il est accessible par la combinaison de touches `Ctrl-Alt-Supr`).  
L'affichage de la commande `top` est donné à la figure suivante.

```
utilisateur@machine:~$ top

top - 22:34:32 up 13:08,  1 user,  load average: 0,57, 0,55, 0,53
Tâches: 328 total,   2 en cours, 326 en veille,   0 arrêté,   0 zombie
%Cpu(s):  2,3 ut,  1,0 sy,  0,0 ni, 96,6 id,  0,0 wa,  0,0 hi,  0,0 si,  0,0 st
MiB Mem :   5879,0 total,   1499,0 libr,   2777,3 util,   1602,7 tamp/cache
MiB Éch:   2048,0 total,   1681,4 libr,    366,6 util.   2723,4 dispo Mem 

    PID UTIL.     PR  NI    VIRT    RES    SHR S  %CPU  %MEM    TEMPS+ COM.     
   3325 john      20   0 1523660  64696  29148 S   5,3   1,1  12:55.64 Xorg     
   3802 john      20   0 5729244 202928  45592 S   4,0   3,4  16:34.15 gnome-s+ 
  22863 john      20   0  829980  52420  38664 S   3,7   0,9   0:05.67 gnome-t+ 
  22224 john      20   0 3379920 430576 156760 S   3,3   7,2   2:59.20 chrome   
  23075 john      20   0   25,5g 261700 100492 S   3,3   4,3   5:51.24 chrome   
    208 root     -51   0       0      0      0 S   2,0   0,0   0:47.74 irq/31-+ 
  22534 john      20   0   39,5g 236192  93140 S   2,0   3,9   1:28.14 chrome   
  22877 john      20   0  363724  85120  20612 S   2,0   1,4   0:15.34 jupyter+ 
  22342 john      20   0 2345324 161496 104324 S   1,0   2,7   2:31.41 chrome   
  22344 john      20   0 1191244  98432  77084 S   1,0   1,6   0:22.37 chrome   
  22469 john      20   0   25,5g 140420  84416 S   0,7   2,3   0:09.15 chrome   
  22521 john      20   0   25,5g 106856  82676 S   0,7   1,8   0:06.59 chrome   
    552 root      -2   0       0      0      0 S   0,3   0,0   1:14.45 gfx      
  21962 root      20   0       0      0      0 R   0,3   0,0   0:02.83 kworker+ 
  22456 john      20   0   25,5g  97076  74044 S   0,3   1,6   0:03.18 chrome   
  23951 root      20   0       0      0      0 D   0,3   0,0   0:03.99 kworker+ 
  24493 root      20   0       0      0      0 I   0,3   0,0   0:01.29 kworker+
```

Si on souhaite interrompre un processus dont on connaît le PID, on peut utiliser la commande [`kill`](https://pubs.opengroup.org/onlinepubs/9699919799/utilities/kill.html), en lui spécifiant les numéros des processus que l'on souhaite terminer.

```
utilisateur@machine:~$ kill 6966 3537
```

La commande ci-dessus va envoyer un **signal de terminaison** aux deux processus listés.  
* Pour les applications graphiques, ce signal est globalement équivalent à fermer la fenêtre principale de l'application.  
* Pour les applications en ligne de commande (qui s'exécutent dans un terminal en bloquant celui-ci), cela correspond à exécuter la combinaison de touche `Ctrl-C`.

Ce signal peut être intercepté par l'application et géré par cette dernière.  
Par exemple, un logiciel de traitement de texte peut, comme lorqu'on ferme la fenêtre, proposer à l'utilisateur de sauvegarder ses fichiers avant de quitter.  
Si on souhaite terminer immédiatement le processus sans que ce dernier puisse intercepter le signal, on peut passer l'option `-9` à la commande `kill`.

```
utilisateur@machine:~$ kill -9 6066 3537
```

Dans ce cas, les processus seront immédiatement terminés, sans qu'ils puissent exécuter la moindre instruction supplémentaire. En particulier, les fichiers non écrits seront perdus.  
Cette option est à utiliser en dernier recours, par exemple lorsque l'application ne *répond* plus, et se comporte de manière anarchique (utilisation importante du processeur ou consommation de mémoire excessive).

## Programmation concurrente
### Processus concurrents
Comme nous l'avons vu, les processus d'un système s'exécutent de manière concurrente : leurs exécutions sont entrelacées.  
De plus, l'ordre dans lequel ils s'exécutent est hors de leur contrôle, car il est décidé par l'ordonnanceur de processus du système d'exploitation.  
Cette façon de fonctionner possède de nombreux avantages. 
* Elle permet l'exécution d'un très grand nombre de programmes sur une machine monoprocesseur. 
* Elle permet aussi d'optimiser les ressources de la machine.  
Par exemple, si un processus est en attente d'entrées-sorties, il est simplement mis en pause et le système peut utiliser le processeur pour effectuer un calcul utile. 
* Et, même si tous les processus sont en attente d'un événement, le système peut alors décider dans ce cas de réduire la fréquence du processeur ou de le mettre partiellement en veille, ce qui permet d'économiser de l'énergie.  
Cet aspect est particulièrement important pour les systèmes mobiles ou embarqués.

Cependant, l'utilisation de systèmes multitâches n'est pas sans problème.  
En effet, lorsqu'un processus est interrompu, il ne s'en *rend pas compte*.  
Dit autrement, lorsqu'un processus est interrompu, il reprendra son exécution exactement dans l'état où il s'était arrêté.  
Tant que ce processus manipule des objets visibles de lui seul (par exemple, des variables allouées sur la pile ou dans le tas), tout va bien.  
Mais si le processus accède à une **ressource partagée**, comme un fichier ou un périphérique matériel, alors de nombreux problèmes peuvent se produire.  

Considérons le programme  [ecritfichier.py](Fichiers/ecritfichier.py) suivant : 

In [ ]:
from os import getpid

pid = getpid()
with open("test.txt", 'w') as fichier:
    for i in range(1000):
        fichier.write(f"{pid} : {i}\n")
        fichier.flush()

Ce programme importe la fonction utilitaire [`getpid`](https://docs.python.org/fr/3/library/os.html#os.getpid) du module [`os`](https://docs.python.org/fr/3/library/os.html).  
Celle-ci ne prend pas d'argument et renvoie simplement l'identifiant du processus dans lequel on se trouve.  
Le programme stocke dans une variable globale son identifiant de processus.  
Puis il ouvre le fichier `test.txt` en écriture (mode `'w'`, qui autorise l'écriture et vide le fichier s'il existe, plaçant le curseur interne en début de fichier).  
Le programme écrit ensuite 1000 lignes de la forme:

    12345 : 0
    12345 : 1
    ...
    12345 : 999

où `12345` est l'identifiant de processus déterminé au début du programme.  
La méthode `.flush()` de l'objet `fichier` assure que les caractères écrits avec la méthode `write` sont effectivement écrits sur le disque.  
En effet, l'écriture sur un disque dur étant coûteuse en temps, les fonctions d'écriture gardent en interne un tableau de caractères *tampon* dans lequel sont accumulés les caractères écrits au moyen de `.write()`.  
Ce n'est que lorsque ce tableau est plein que les caractères sont effectivement écrits sur le disque et le tableau est vidé. 

Que se passe-t-il maintenant si on lance trois copies de ce programme simultanément?

```
utilisateur@machine:~$ python3 ecritfichier.py & python3 ecritfichier.py & python3 ecritfichier.py &

[1] 82208
[2] 82209
[3] 82210
[1]  Fini python3 ecritfichier.py
[2]- Fini python3 ecritfichier.py
[3]+ Fini python3 ecritfichier.py
```

L'utilisation du caractère `&` à la suite d'une commande bascule directement ce programme en arrière-plan et rend directement la main au terminal.  
On lance donc trois fois le programme.  
Le *shell* indique, dans la console, les identifiants des processus correspondants (ici respectivement `82208`, `82209` et `82210`).  
Les trois lignes contenant `Fini` sont écrites lorsque les programmes se terminent.  

Si on observe le contenu du fichier `test.txt`, on peut observer des lignes telles que :

    ...
    82209 : 840
    82209 : 841
    82208 : 842
    82210 : 843
    ...

On a donc l'impression que certaines lignes ont été écrites par certains processus uniquement.  
La réalité est plus complexe.  
Lors de l'ouverture du fichier, chaque processus initialise dans sa mémoire une variable de position (le curseur) contentant le décalage par rapport au début du fichier.  
Lorsque des caractères sont écrits au moyen de `.write()` le curseur est avancé d'autant de caractères.   

Un enchaînement d'exécutions de processus provoquant les écritures est représenté sur la figure suivante :

<div style="text-align: center">
   <img alt="Commutations de contexte entre processus" src="Images/concurrence-1.png" > 
</div>

Cette figure représente dans trois colonnes les états des trois processus, en particulier les valeurs internes du curseur dans le fichier `test.txt` et les chaînes qui sont écrites, lorsque le processus a la main.  
La figure représente l'instant où le processus `82208` a la main. Ce dernier est à une certaine position dans le fichier (`9970`) et y écrit deux lignes.  
Il est alors interrompu par l'ordonnanceur qui provoque un changement de contexte et donne la main au processus `82210`.  
Ce dernier se trouvait à la même position dans le fichier, et donc au même indice de boucle (car les trois processus sont partis de l'indice `0` et écrivent exactement les mêmes nombres de caractères dans le fichier).  
Le processus `82210` écrit aussi deux lignes avant d'être interrompu.  
Le processus `82209` passe alors en exécution et peut écrire trois lignes.  
Comme c'est le dernier processus à écrire les lignes $840$ et $841$ ce sont ses écritures qui se retrouvent dans le fichier final.  
L'ordonnanceur rend la main au processus `82210` qui se retrouve dans l'état où sa variable de curseur vaut `9994` et sa variable `i` de boucle vaut `842`.  
Il va donc procéder à l'écriture de la chaîne `82210 : 842\n` dans le fichier, écrasant, par là même, la ligne qui avait été écrite à cette même position par le processus `82209`.  
Il est important de noter que cet *entrelacement* des instructions des trois processus est *aléatoire* ou plus exactement **non déterministe**. Si on réexécute les trois programmes, l'ordonnanceur peut décider de changer de contexte à d'autres moments, en fonction de divers paramètres (nombre de processus total en cours d'exécution, valeur des horloges, etc.) et on obtiendra un fichier différent.

|            | 
|:----------:|
| ![Ordonnancement](Images/ordonnanceur-1.png) |
| ![Ordonnancement](Images/ordonnanceur-2.png) |

|            | 
|:----------:|
| ![Ordonnancement](Images/ordonnanceur-3.png) |
| ![Ordonnancement](Images/ordonnanceur-4.png) |

|            | 
|:----------:|
| ![Ordonnancement](Images/ordonnanceur-5.png) |
| ![Ordonnancement](Images/ordonnanceur-6.png) |
| ![Ordonnancement](Images/ordonnanceur-7.png) |

|            | 
|:----------:|
| ![Ordonnancement](Images/ordonnanceur-8.png) |

|            | 
|:----------:|
| ![Ordonnancement](Images/ordonnanceur-9.png) |
| ![Ordonnancement](Images/ordonnanceur-10.png) |

|            | 
|:----------:|
| ![Ordonnancement](Images/ordonnanceur-11.png) |

|            | 
|:----------:|
| ![Ordonnancement](Images/ordonnanceur-12.png) |

<!--

1
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7Vpdc6o8EP41XNaBRBAuq56vizP1nV68H3eURM0UwRNC1f76NyEJ3x7Ug7Y6dpwpWTabbPbJstmNASer7Tfqr5c%2FY4RDA5hoa8CpAYALgCF%2BJtpJwtA0JWFBCZIkqyA8k3esiJotJQgnFUYWxyEj6yoxiKMIB6xC8ymNN1W2eRxWR137C9wgPAd%2B2KT%2BTRBbarVGBf07JoulHtlyPPlm5WtmpUmy9FG8KZHgFwNOaBwz%2BbTaTnAo1k6vi%2Bz3dc%2FbfGIUR%2ByQDglKbfs%2F8r5Mvz1b3%2B2%2F3jdR8qCkvPlhqhRmOGEDtmUGcEIud%2FxC%2BdNCPGkKIm9CHbZTa%2BT8SoUOY4a37MEPySIy4CPnCPFcdVMMWtBgMNCy%2BHQzcX84ArdrxB6SDD3ihWWut61D6xEEf4aaMKaVIQwATf43n5dJLVM5dEC%2BAUyXM0g2d2iWNJeTqE7soxakaeabm8kVDQgqAwHuOtbiMYoZ%2FzfeLAnDz2s%2FELQNd7uctmSrkLcsId4PXhc0TiP0lLKQRFjRkU9fn3gvwoQvNgemzYnSMFPLycd9w5RrtNfJWLnr4i4fxyvM6I6zbLV7lz122h2CgS0pm8J7Wtq5L8ueEypGX3nsRS68cGr8Qfm1I3wcaPi4GY0DnCQ4pftNchlKzdCZ1TBSBuuwcw%2FmAmbNXvpjVbJWzlOxlmbs3VqwYa0nivjH3Y%2BCT2CvM9ldb%2FC1BObpe1xtZ765IQC2azpD23Is4Jmwp%2F1d2%2BBuc3fDURtevHPhZdjAS2N1cYQeRSjIW0HoJwkJqotZ7Dr%2BmR7jLWH%2FlJ7%2FzdbTVq3ptvRqutONiKuSdRp4rqsJpZ6iWXTNWpW%2BM0wJXw9MFVHqgFEjOK0ZiusZpzTAv1kgtR7MpwvMukLBpuFLlrVbDKtpFIc%2BI2%2FV6bYZW40wi0kWgSlcQa%2FmiJwaXqSaqlc5yq0JqgM0%2F9ZoQXIdGoIy7OVqnw5Hu3c45tCqwgp04epwCHVCA34kNIb1b5R9pdBwLuOpRge6KquKp1F%2FeOp0SfBA3IG7S%2BoBd14L7iqn4NNPt83QZvZjWpx2ZRrnbHkELShZ%2B1GrpCJKelAnfSGPRIQRPzz%2BiCbHuehRsUfVZpOxtsyNaHTDxgoJjvJ9VE%2FJXb12qxjhXDtjAo1HL6CEpRTfjIo3jM0gpTJ1YmaCTM8bmTejHMnVquZqD9eq1eLVVO%2FerI%2BIszpO%2FnMShpM8bQ2RjV00FGvAaPyKS29c8AKdntJ6Tj0Gh8080Z6s3rnO%2FToB9SFhjendw5pPoto9rLki1e5hzZWreMPYvIc1nyismbsBDkTyqxHWvLj20C7yUX9WzRh9vrCmeSOjt7CmHsS4FwhiLl5cv3uE2%2FQIyMfuvNUjOIGLX%2Bb9eAQwtNuLDV0eYXQ2hwCOKhtEsagF768ZCPuoW3aWbpfWUtx9%2Bsqnu7dM0Fl2ai9KgN8XJXqsLRxc1NqHhMtUFxo3ZU6uLtSP5nVBZ64ugOaFjTs%2BD8hbdOPTND4Qn9CuwgrUL%2FycjM%2B6pzw3Po%2B7INIbPhXMrGNgdoFafyc%2BvQPh6d0EOhuJTQf2hE7eLO52S%2Fbigjz88j8%3D
2.
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7VpLc%2BI4EP41PoayJWzsYwLz2MNU2MphHzfFFqCKsRhZDpBfv5It%2Bc0YGEOAJUVVrHar5VZ%2FarW6ZcDxcvONodXiBw1waAAz2BhwYgDgAmDInxlsM8LQNDPCnJEgI1kF4YV8YEXUbAkJcFxh5JSGnKyqRJ9GEfZ5hYYYo%2Bsq24yG1VFXaI4bhBcfhU3qXyTgC63WqKB%2Fx2S%2B0CNbjpe9WSLNrDSJFyig6xIJfjHgmFHKs6flZoxDOXd6XrJ%2BX3e8zT%2BM4Yjv0yEOEtv%2Bl3wskm8v1nf7z491FD8oKe8oTJTCHMd8wDfcAE4o5D69MvE0l0%2BaEpB3qQ7fqjlyfiZShyeON%2FwBhWQeGfBRcIR4propBi1oMBhoWeJzU3G%2FOYKwa8Qf4hQ98oVlrjatQ%2BsRJH%2BKmpCyyhAGgKb4m83KpJZP2XdAsQBMVzBkbO7QLGmefUT1w65jQtp1s86j2y4Vmkj9%2Fw0IKgMB4XNW8jGiXPx7Wi8Ixy8r5EvaWvhrQVvwZShalhSP%2FLc5o0kUPCc8JBFW9ACxt2fRi3DpxM2BaQtihq2J5eTjvmMmNNrpnazc54m9AtMl5mwrWDZ6X8h6bLUfBQM7o6wLt2vpXWFRdrlQMSLl6ue58MIbigflEA9wjqDhHKeM%2BjiOccJ2m%2BQ8lJqhU6vhQBmsw849mAuYNXvpXa5krZynYi3N2Lu1YMNazywQUQGK%2FAuw14nsrhf4KgPm8WtcLWexuCEAtms6Q9tyLOCZsKf1XVvgbnN1w1EbXrxT4WXYwEtjdnEUPMoYUrT8EMUx8auTWaw6sQc%2B4Q3hf5ee%2F0nn01atyab0arLVjUioknYaeK6rCaWesll0TVuVvlPMiJgPzBQx0wEHjai2ZiihJ02Yj38xQWo%2BOGJzzLtiyKbhS5a1WwyraQyHiJP36ue2GVuNMKUkjVQUrqBXc0RODS%2BZmqpXOTyuCaoDNN9rtKBsHhqCUuzlah8PR7t3OObQqsIKdOFqfwh1QgN%2BJjSG9T3KvlJoOOfxVKM9XZVVxdOoPzx1uiS4J%2B7A3SX1gDuvBXeV0%2BLxx%2BJmaDP9Y1IcJbP8z8kSEFpQvEJRq6QiSnpQJ2Ipj0SEExQefkTLxjnrUbFH1abjJ22ZG9Hoho0VEhzl66iey7t67ZY0wLl2xhgaj57PCE8YvhkVbxibfsKy1ImZCjI9b2TejHIkV6ua5N1fq1aLV%2FOoO7M%2BMs7qOPnPSBiO8%2FQuDGzsBkM5B5zRN1x644JX6PSU1nPqMThs5ol2ZPVOde7XCahPCWtM7x7WXIhq97DmilS7hzVXruINY%2FMe1lxQWDNzfezL5FcjrHl17aFd5KN%2Br5oxurywpnmVo7ewph7EuGcIYs5eXD%2BpR3DBUWvnEpUrewTrKK3OfNBB2J21egTHd%2FHrrB%2BPAIZ2e7GhyyOMTuYQwEFlg4jKWvDumoG0j7qeZ%2Bl2aS7lHaGv4nN3lgk6y07tRQnw66JEj7WFvYtau5BwnupC46bM0dWF%2BtG8LujE1QXQvLBxx%2BceeYtufJrGJ%2BIT2lVYgfqFn6PxWfeUp8bnYRdEesOngpl1CMzOUOvvxKe3Jzy9m0BnI7HpwJ7QKZrFpfCMvbhZD7%2F8Bw%3D%3D

3.
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7Vpbc6o6FP41PNYJCSA8Vt23hzP1TB%2F25Y1C1EwRPCFU7a8%2FgSTc3ahFq45OZ0oWyQpf1peVlaxoaLzcfKPuavFP5ONAg8DfaGiiQWhDqKV%2FwN8KgQGAEMwp8YVILwTP5B1LoaqWEB%2FHlYosigJGVlWhF4Uh9lhF5lIaravVZlFQ7XXlznFD8Oy5QVP6k%2FhsoWANC%2Fl3TOYL1bNuOeLN0lWVJZJ44frRuiRCXzQ0plHExNNyM8ZBOnZqXES7rzve5h9Gccj2aRD7iWn%2BIe%2BL5Nuz%2Ft38930dxg9Sy5sbJBIwwzEbsA3ToBVwvaMXyp%2Fm6ZOS%2BOQthcO2coys%2F5IUw4jhDXtwAzIPNfTIawR4JpvJCkrRYDBQuvjnZuo%2B2AO3a8ge4ow96QsdrDatXase0voZa4KIVrrQIALAcVKWFqKWT9m3Q1tQGYhqtgFKyMVHVD%2FsEgdklv1asQG7hE2%2FUGxNCvf0JVfUIax0BLkzWqWPYcT4v9F6QRh%2BXrleKltzR85lC7YMeElP1bve65xGSeg%2FJSwgIZZy36WvT7wVYal3BwNgcqEwzES38n7fMOWIdrotPXeGfBHB0RIzuuVVNmrBEC22ysHCgSkk68If62q5WJR9MZIVXbkGzHPlhZvkD9JTHuA1YcNrTmnk4TjGCd1tkvNIaobOrIZ9abAOO%2FdgLghq9lLLX8laeZ2KtVTF3q2FGtZ6oj4PF9zQuwB7ncjuaoKvBDGPn%2BNyOvPJjSA0bWAZpm7p0AGop%2Fldm%2BB2c3ajYRtfnFPxxWjwpTG6OPQf0%2BCSl7zAjWPiVQezmHV8cRzhDWG%2FSs%2B%2Fs%2FE0ZWmyKb2abFUh5FCyRgPHtpWg1DItFk2zUqXtFFPCxwNTKRQYsN8Id2uG4jijhHr4LwMkx4O5dI5ZV3DZNHzJsmaLYZWM4sBl5K36uW3Glj1MI5KFMJJXyKk5IqvGFwFTtirHzTVFdYLma41SJMahoSjjXg77eDqavdMxp1aVVrCLV%2FtTqJMa6DOpYdTXKPNKqWGdx1MN93RVepVPw%2F741OmS0J68g3eX1APvnBbeVbaRx%2B%2BXm6HN9Mek2GOKHefJTiaUonjlhq2aiijpQW6VU30kJIy4weFbNNHPWbeKPUKbjkfKMjeC6IaNFRAc5vOofsh39eiWkY9zdNoYaY%2BORwlLKL4ZiDfMTS%2Bh4ugEZIqA4zjGzYAjOSzbgEeharV49YB156lPGmd17PxnJAjG%2Bbkv8l1sz9IoMWY0esWlN5Zn45dZ3t%2BHtv1WPQZHzXOiHad6p9r3qwOozwlrnHtYcyHQ7mHNFUG7hzVXDvGGudkMa4bgZsCVw5rjUJ03rJnZHvZaw5oX2zTM4jzqY9mM4eWFNc07Hr2FNbUgRge7bXu9yfW7R7hNj%2BCb2PaNNo9gwxdk9XR%2FARpme7KhyyMMT%2BYQ4EFpgzBKc8G7cwapfeS9PV2VS2MJ%2BO8r%2F9ydaYLOtFN7UgL%2BPSnRY25h76TWLiacJ7vQuClzdHahvjWvKzpxdgE2L2zc%2BbnHuUU3P4H2ifxEZpVWsH7h52h%2B1j3lqfl52AWR3vgpaaYfQrMz5Po7%2BensSU%2FnJtjZONi0UE%2Fs5MXitrioXly5R1%2F%2BBw%3D%3D

4.
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7Vpbc6o6FP41PNYJCSA8Vt23hzP1TB%2F25Y1C1EyReEKo2l9%2FAiTc3ahFq44dZ0oWyQpf1peVlaxoaLzcfGPuavEP9XGgQeBvNDTRILQh1JIf8LeZwAAgE8wZ8TORXgieyTuWQlUtJj6OKhU5pQEnq6rQo2GIPV6RuYzRdbXajAbVXlfuHDcEz54bNKU%2Fic8XCtawkH%2FHZL5QPeuWk71ZuqqyRBItXJ%2BuSyL0RUNjRinPnpabMQ6SsVPjkrX7uuNt%2FmEMh3yfBpEfm%2BYf8r6Ivz3r381%2F39dh9CC1vLlBLAFzHPEB33ANWoHQO3ph4mmePCmJT94SOHwrx8j6L04wjDje8Ac3IPNQQ4%2BiRoBnspmsoBQNBgOlS3xuqu6DPQi7hvwhStmTvNDBatPateohqZ%2ByJqCs0oUGEQCOk7C0ELV8yr4d2hmVQVbNNkAJefYR1Q%2B7jgFpx6ZfKLYmhXv6kivqEFY6gsIZrZLHkHLxb7ReEI6fV66XyNbCkQvZgi8DUdIT9a73Omc0Dv2nmAckxFLuu%2Bz1SbQiPPHuYABMIcwMM9GtvN83zASinW5Lz52hWEQwXWLOtqLKRi0YWYutcrBwYGaSdeGPdbVcLMq%2BGMmKrlwD5rnywk2KB%2BkpD%2FCasOE1p4x6OIpwzHab5DySmqFTq2FfGqzDzj2YC4KavdTyV7JWXqdiLVWxd2uhhrWemC%2FCBTf0LsBeJ7K7muCrjJjHz3E5ncXkRhCaNrAMU7d06ADU0%2FyuTXC7ObvRsI0vzqn4YjT40hhdHPqPSXApSl7gRhHxqoNZzDqxOI7whvBfpeff6XiasjTZlF5NtqoQCihpo4Fj20pQapkUi6ZpqdJ2ihkR44GZFGYYsN8Id2uGEjhpzDz8lwGS48FdNse8K7hsGr5kWbPFsErGcOBy8lb93DZjyx6mlKQhjOQVcmqOyKrxJYMpW5Xj5pqiOkHztUYpysahoSjlXg77eDqavdMxp1aVVrCLV%2FtTqJMa6DOpYdTXKPNKqWGdx1MN93RVepVPw%2F741OmS0J68g3eX1APvnBbeVbaRx%2B%2BXm6HN9Mek2GOK2NE%2B4cmEUhSt3LBVUxElPcitcqKPhIQTNzh8i5b1c9atYo%2FQpuORssyNILphYwUEh%2Fk8qh%2FyXT26JfVxjk4bI%2B3R8RjhMcM3A%2FGGuenFLDs6Aaki4DiOcTPgSA7LNuBRqFotXj1g3Xnqk8RZHTv%2FGQmCcX7ui3wX27MkSow4o6%2B49MbybPwyy%2Fv70LbfqsfgqHlOtONU71T7fnUA9TlhjXMPay4E2j2suSJo97DmyiHeMDebYc0Q3Ay4clhzHKrzhjUz28Nea1jzYpuGWZxHfSybMby8sKZ5x6O3sKYWxOhgt22vN7l%2BUo9gH7cluERwZY%2BgH4XqzBsdE9u%2B0eYRbPiCrJ7uL0DDbE82dHmE4ckcAjwobRDSJBe8O2eQ2Efe29NVuTSWQPx9FZ%2B7M03QmXZqT0rAvycleswt7J3U2sWE82QXGjdljs4u1LfmdUUnzi7A5oWNOz%2F3OLfo5ifQPpGfyKzSCtYv%2FBzNz7qnPDU%2FD7sg0hs%2FJc30Q2h2hlx%2FJz%2BdPenp3AQ7GwebFuqJnaJY3BbPqhdX7tGX%2FwE%3D

5.
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7Vpdc%2FIqEP41uayTQBKTy6rv18WZeqYX78ddmqAyjcFDSNX%2B%2BgMJ5Ns3aqNVR6czhc2ysOzDsrBocLzcfKPeavEPCVCoAT3YaHCiAeAAoIk%2FPdhmBFPXM8Kc4iAjGQXhGb8jSVRsCQ5QXGFkhIQMr6pEn0QR8lmF5lFK1lW2GQmrva68OWoQnn0vbFJ%2F4oAtlFrDgv4d4flC9WzYbvZl6SlmqUm88AKyLpHgFw2OKSEsKy03YxSKuVPzkrX7uuNrPjCKIrZPgzhILOsPfl8k356N79a%2F7%2BsofpBS3rwwkQozFLMB2zAN2CGXO3qhvDQXJUUJ8JtQh23lHNn%2FJUKHEUMb9uCFeB5p8JFzhGgmm0kGJWgwGChZfLipuA%2F2wO0asYc4RY%2F4YOirTWvXqgfBn6ImJLTShQbgbAb5r0xqGcq%2BHfIFoHNE6BmbY%2BolzbNBVAd2mRMifq26iWWa62ZcqG5NCPc0kivqEFQ6AtwZrUQxIoz%2FG60XmKHnlecL2po7ck5bsGXIa4YQ7%2Fmvc0qSKHhKWIgjJOmBR1%2BfeCvMhHfXB7rFiZlhJoad9%2FuGKNdop9sycmfINxFElojRLWfZqA0ja7FVDhYMrIyyLvyxobaLRdkXQ8noyT1gngsv3CQvSE95gNcEDa85pcRHcYwSutsk56HUDJ1aDQXSYB127sFcQK%2FZS21%2FJWvlPBVrKcberQUb1nqiAQ8XvMi%2FAHudyO5qga8yYB6%2FxuVy5osbAmA5um1ahm0AV4c9re%2FaAneaqxsO2%2FDingovZgMvjdlFUfAogkte80MvjrFfncxi1fHNcYQ2mP0qlX%2Bn82nJ2mRT%2BjTZqkrEVUkbDVzHUYRSS1Etmqa1StspopjPB6KSmOmAgka4WzMU15Mk1Ed%2FmSA5H8yjc8S6gsum4UuWtVoMq2gUhR7Db9Xhthlb9jAlOA1hJK6gW3NEdg0vmZqyVTlurgmqAzTfa5SgbB4aglLs5WofD0erdzjm0KrCCnThan8IdUIDfiY0zPoeZV0pNOzzeKrhnq7KqOJp2B%2BeOl0S3BN34O6SesCd24K7yjHy%2BPNyM7SZ%2FpgUZ0weOzonvJlQguKVF7VKKqKkB3lUFvJwhBn2wsOPaFk%2FZz0q9qjadDxSlrkRjW7YWCFGUb6O6pd8V6%2FdkgQo104bQ%2B3R9SlmCUU3o%2BINY9NPaHZ1oqeCdNd1zZtRDudqOSY4SqtWi1cvWHfe%2Bog4q%2BPkP8NhOM7vfWHgIWcmosSYUfKKSl9s30Evs7y%2FDx377XoMDpv3RDtu9U517lcXUJ8S1hj6bmtfIqpv2Bvdw5orUu0e1ly5ijeMzXtYc0lhjYWcwNRawhoHvEC7r2zl8PLCmuYbj97CmvrdjHuGIObsyfWTeoShftTauUTlyh7hOK3O6xFmjo%2F81oPOi2OZVnFD%2FbGEuGm1Jxu6PMLwZA4BHJQ2iIjIBe%2FOGQj7yHd7hqqX5lLnv698uDvTBJ1pp%2FakBPh7UqLH3MLeSa1dSDhPdqHxUubo7EL9aF4XdOLsAmg%2B2Ljjc497i2586ton4hNaVViB%2BoOfo%2FFZ95SnxudhD0R6w6eEmXEIzM6Q6%2B%2FEp7snPN2bQGfjYtOGPaGTV4vX4hl78eQefvkf

6.
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7Vpdc%2FIqEP41uayTQBKTy6rv18WZeqYX78cdTVCZxuAhpGp%2F%2FSEJ5Ns3aqNVR6czhc2ysOzDsrBocLzcfGNotfiH%2BjjQgO5vNDjRAHAA0JI%2F3d9mBFPXM8KcET8jGQXhmbxjSVRsMfFxVGHklAacrKpEj4Yh9niFhhij6yrbjAbVXldojhuEZw8FTepP4vOFUmtY0L9jMl%2Bong3bzb4skWKWmkQL5NN1iQS%2FaHDMKOVZabkZ4yCZOzUvWbuvO77mA2M45Ps0iPzYsv6Q90X87dn4bv37vg6jBynlDQWxVJjjiA%2F4hmvADoTc0QsTpXlSUhSfvCXq8K2cI%2Fu%2FONFhxPGGP6CAzEMNPgqOAM9kM8mgBA0GAyVLDDcV98EehF1D%2FhCl6Ek%2BGPpq09q16iHhT1ETUFbpQgNwNoPiVya1DGXfDsUC0AUi9IzNMfWS5tkgqgO7jglp1824UN2aEO5pJFfUIah0BIQzWiXFkHLxb7ReEI6fV8hLaGvhyAVtwZeBqBmJeOS9zhmNQ%2F8p5gEJsaT7iL0%2BiVaEJ95dH%2BiWIGaGmRh23u8bZkKjnW7LyJ2h2EQwXWLOtoJlozaMrMVWOVgwsDLKuvDHhtouFmVfDCUjknvAPBdeuElRkJ7yAK8JGl5zyqiHowjHbLdJzkOpGTq1GvalwTrs3IO5gF6zl9r%2BStbKeSrWUoy9Wws2rPXEfBEuoNC7AHudyO5qga8yYB6%2FxuVyFosbAmA5um1ahm0AV4c9re%2FaAneaqxsO2%2FDingovZgMvjdnFof%2BYBJei5gUoiohXncxi1YnNcYQ3hP8qlX%2Bn82nJ2mRT%2BjTZqkooVEkbDVzHUYRSy6RaNE1rlbZTzIiYD8wkMdMB%2B41wt2YooSeNmYf%2FMkFyPjhic8y7gsum4UuWtVoMq2gMB4iTt%2Bpw24wte5hSkoYwElfQrTkiu4aXTE3Zqhw31wTVAZrvNUpQNg8NQSn2crWPh6PVOxxzaFVhBbpwtT%2BEOqEBPxMaZn2Psq4UGvZ5PNVwT1dlVPE07A9PnS4J7ok7cHdJPeDObcFd5Rh5%2FHm5GdpMf0yKM6aIHZ0T3kwoQdEKha2SiijpQR6VE3kkJJyg4PAjWtbPWY%2BKPao2HY%2BUZW5Eoxs2VkBwmK%2Bj%2BiXf1Wu3pD7OtdPGUHt0PUZ4zPDNqHjD2PRill2d6Kkg3XVd82aUI7lajgmO0qrV4tUL1p23Pkmc1XHyn5EgGOf3vtBH2JklUWLEGX3FpS%2B25%2BCXWd7fh479dj0Gh817oh23eqc696sLqE8Jawx9t7UvEdU37I3uYc0VqXYPa65cxRvG5j2suaSwxsKOb2otYY0DXqDdV7ZyeHlhTfONR29hTf1uxj1DEHP25PpJPYJz3Nq5ROXKHsE4SqvzeoSZ42Gv9aDz4limVdxQfywhblrtyYYujzA8mUMAB6UNQprkgnfnDBL7yHd7hqqX5lIXv69iuDvTBJ1pp%2FakBPh7UqLH3MLeSa1dSDhPdqHxUubo7EL9aF4XdOLsAmg%2B2Ljjc497i2586ton4hNaVViB%2BoOfo%2FFZ95SnxudhD0R6w6eEmXEIzM6Q6%2B%2FEp7snPN2bQGfjYtOGPaFTVIvX4hl78eQefvkf

7.
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7Vtdc7I6EP41XNaBRBAuq75fF2fqmV6cj7sUomaKxBNC1f76EyDhmxds0apjpzMlS7LJ7j7ZbHapBmeb%2FQ%2BGtus%2FqId9DejeXoNzDQAbAC3%2B1b1DShjrekpYMeKlJCMnPJN3LImqW0Q8HJY6ckp9TrZlokuDALu8REOM0V2525L65Vm3aIVrhGcX%2BXXqX8TjayXWJKf%2FxGS1VjMblpO%2B2SDVWUoSrpFHdwUS%2FKbBGaOUp0%2Bb%2FQz7se6UXtJx31veZgtjOOB9BoReZJr%2Fkvd19OPZ%2BGn%2B%2Bb4LwgfJ5Q35kRSY45CP%2BJ5rwPIF3%2BkLE0%2Br%2BElRPPIWi8MPUkfWf1Esw5TjPX9APlkFGnwUPXy8lMNkB8VoNBopXmK5CbtPziDsGvCHMEFP%2FMLQt%2FvGqdUMcf8ENT5lpSk0AJdLKH6KpIal9J1QbABdIEJPu9ljvSB5uojywq5DIc2yGeeRrU2ENkZfoQowqCqOlaC%2BZ5vnO7FBupcBShMB4SK38WNAufgz3a0Jx89b5Ma0nTheBG3NN75oGTF75L6uGI0C7yniPgmwpHuIvT6JUYTHZ44%2B0k1BTLU3N6xs3jfMhEStztTIXLQ42jDdYM4OosteHWPpiINy%2B2BkppRdfkoY6hBbF08IKDsieTKtMua58xYP0n8f4ctBzZcvGHVxGOKItZvkPJSKoROrYU8arMPOA5gL6BV7qUO5YK2sT8laquPg1oI1az0xTwQxKHAvwF4nsrva4NsUmB%2Ff43I7i80NATBt3RqbhmUAR4cD7e%2FKBrfruxtOmvDinAov4xpeatrFgfcYh7yi5fooDIlbVma%2B68Q5NcV7wv8uPP%2BT6NOUrfm%2B8Gp%2BUI1AiJIMGjm2rQiFkXEzH5q0SmMXmBGhD8wkMZUBe7UgvGIoISeNmIt%2FoyCpD47YCvOukLdu%2BIJlzQbDKhrDPuLkrbzcJmPLGRaUJNGExBV0Ko7IquAlFVOOKkbzFUZVgGZnjWKU6qHGKMFeJvbH4WgODscMWmVYgS5c9YdQJzTgV0JjXD2jzCuFhnUeTzXp6aqMMp4mw%2BGp0yXBnrgDd5c0AO6cBtyVbnRDXqwWv%2Bb5dU%2FEjvYJ8yWKUbhFQSOnPEp6kLfWmB8JCCfIP%2F6Kls5z1qvigKItZlNlmRuR6IaN5RMcZPuomnq8euk21MOZdNoMao%2BOywiPGL4ZEW8Ym27E0tSJnjDSHccZ34xwJBOrnKzsL1Wjxcu5ztasTxxnddz8l8T3Z1kKFnoI28s4Sgw5o6%2B48MZybfyyzOb71LXfqsbgsJ4nasnqnererxJQXxLWGHq7tS8R1Tfsje5hzRWJdg9rrlzEG8bmPay5pLDGxLY31hrCGhu8QGuoauXk8sKa%2Bpcng4U11dyMc4Ygpm%2FJ%2FRI3zd0jXJBHWNoudhsvOi%2B2OTbzDPXnCuJjs7nY0OURJidzCOCoskFA41pwe80gto%2F8mtBQ7YIudfHzXSy3tUzQWXZqLkqA3xclBqwt9C5qtSHhPNWF2pcyH64uVK%2FmVUYnri6A%2Bgcbd3z2yFt041PXvhCf0CzDClQ%2F%2BPkwPque8tT4PO4DkcHwKWFmHAOzM9T6O%2FHp9ISncxPorCU2LTgQOkUz%2F4Y97Z7%2FIwD89j8%3D

8.
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7Vtdc7I6EP41XNYJCSBcVn2%2FLs7UM704H3cUomaK4Amhan%2F9SSDhmxdt0apjpzNNlmST3X2y2WRTDU3Xux%2FU3az%2BiHwcaBD4Ow3NNAhtCDXxC%2Fx9RjAAyAhLSvyMpBeEZ%2FKOJVE1S4iP40pDFkUBI5sq0YvCEHusQnMpjbbVZosoqI66cZe4QXj23KBJ%2FYv4bKXEGhf0n5gsV2pk3XKyL2tXNZaSxCvXj7YlEvqmoSmNIpaV1rspDoTulF6yft87vuYTozhkh3SI%2FcQ0%2FyXvq%2BTHs%2F7T%2FPN9G8YPksubGyRSYIZjNmI7pkEr4HwnL5SXlqKkKD55E%2BKwvdSR9V8iZJgwvGMPbkCWoYYeeYsAL2Q32UAxGo1GihefbsrukyNwu4bsIU7RIz7oYLNrHVqNINqnqAkiWhlCg2ixQPynTGqZyqED8gUAOCJA1sw2QEnybBLViV2HQtpl088jW5cIXYyGVAUAjiM8WIsqhMfKVQEHVcWxEjTXbPt4JzZI%2FzRgZSDIXeRGFMOI8T%2BT7Yow%2FLxxPUHb8u2F01ZsHfCaLti73uuSRknoPyUsICGWdN%2Blr0%2B8F2FizwEjYHJipr2ZbuXjvmHKJep0pnruovnWhqM1ZnTPm%2BzUNpb12Cu3D0dmRtkWu4SuNrFVeYdAsqErd6Zlzrxw3rwg%2FfcRvhw2fPmcRh6OY5zQbpOch1IzdGo17EuD9dh5AHNBULOX2pRL1srbVKylGg5uLdSw1hP1eRDjht4F2OtEdlcLfJMB8%2BNrXC5nvrgRhKYNLMPULR06AA20vmsL3G6ubjRuw4tzKrwYDbw0tItD%2F1GEvLzmBW4cE6%2BqzGLV8X1qgneE%2FV0q%2F5Pq05S12a70abZXlZCLknYaObatCKWeolp0TWuVvnNMCdcHppKYyYD9RhBeMxSXM0qoh3%2BjIKkP5tIlZn0hb9PwJcuaLYZVNIoDl5G36nTbjC1HmEckjSYkrpBTc0RWDS%2BZmLJXOZqvMaoDNN9rFKNMDw1GKfZysT8OR3NwOObQqsIK9uHqcAj1QgN9JTSM%2Bh5lXik0rPN4qvGBrkqv4mk8HJ56XRI6EHfw7pIGwJ3TgrvKiW7Ig9X816w47vHY0T7hfYliFG%2FcsJVTESU9yFOr4EdCwogbHH9Ey8Y561FxQNHm04myzI1IdMPGCggO83VUv3q8eunWkY9z6bQp0h4djxKWUHwzIt4wNr2EZlcnIGUEHMcxbkY4kotVvaw8XKpWi1fvOjtvfUSc1XPyX5AgmOZXsMh3sb0QUWLMaPSKS18sz8Yvi3y8Tx37rXoMjpr3RB23eqc696sLqK8Ja5x7WHMhot3DmisS7R7WXLmIN4zNelijAwCsm5GuHNegK4hrFraHvda45sU2DbO4kPpcOmN8eXFN8%2BnJYHFNLYrRQbdtz5jsvtxFcz%2FpXJBH8E1s%2B0abR7DhC7IGesAADbM929DnEcYncwjwqLxBGIlkcHfSQNhHPifUVb2kS77pge98up15gt68U3tWAv4%2BKzFgcuHgrFYXEs6TXmg8lflweqF%2BNq8zOnF6ATZfbNzxecDFRT8%2BgfaF%2BERmFVaw%2FuLnw%2Fise8pT4%2FO4FyKD4VPCTD8GZmdI9vfi0zkQns5NoLNxs2mhgdDJq8Uj9qx58Z8A6Nv%2F

9.
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7Vvfc6I%2BEP9reKwDCSA8Vr1fDzf1pg933%2B8bhaCZInghVO1ffwkk%2FOZQD606djpTsiQbdveTzWY3VeB0tf1CnPXye%2BShQAGqt1XgTAHAAkDhv6q3ywi6qmaEBcFeRtIKwjN%2BR4IouyXYQ3GlI42igOJ1lehGYYhcWqE5hESbajc%2FCqqzrp0FahCeXSdoUn9ijy6lWOOC%2FhXhxVLOrJl29mblyM5CknjpeNGmRIKfFDglUUSzp9V2igKuO6mXbNznjrf5hxEU0n0GxF5iGP%2Fj92Xy5Vn7avx434Txg%2BDy5gSJEJiimI7olirADBjfyQthTwv%2BJCkefuPi0J3Qkfk74TJMKNrSByfAi1CBj6xHgHwxTHSQjEajkeTFPjdl948zMLuG9CFO0cNfaOp62zq1nIH3T1ETRKQyhQKg70P2Uya1fMq%2BE7IFoDJEqFk3S1dLkmcfUf2w61BIu2zaeWTrEqGL0ZCqUNmP77erwiqpAgyqikMlaK7Z9vlObJD%2BzwCViQBzkWv%2BGEaU%2FZlslpii57XjctqGbS%2BMtqSrgLU0zt5xXxckSkLvKaEBDpGgew55fWKjMOV7jjpSDUbMtDfTzHzeN0SYRJ3OVMtdNNvaULRClOxYl63cxrIRO%2Bn2wcjIKJtil9DkJrYs7xBQdHTEzrTImRfOmz0I%2F32ALwcNXz4nkYviGCWk2yTnodQMnVoNecJgPXYewFxArdlLbsola%2BV9KtaSHQe3FmxY64l4LIhxQvcC7HUiu8sFvs6AefwaF8uZLW4IgGGppm5opgZsFQ60vmsL3Gqubjhuw4t9KrzoDbw0tItC75GHvKzlBk4cY7eqzGLVsX1qgraY%2Fio9%2F5fq0xCt2bb0araTjZCJkg4a2ZYlCaWRvFkMTVuVsXNEMNMHIoKYyYC8RhBeMxSTM0qIi%2F6iIKEP6pAFon0hb9PwJcsaLYaVNIICh%2BK36ue2GVvMMI9wGk0IXEG75ojMGl4yMcWocjRfY1QHaL7XSEaZHhqMUuzlYh8PR2NwOObQqsIK9OFqfwj1QgN%2BJDT0%2Bh5lXCk0zPN4qvGerkqr4mk8HJ56XRLcE3fg7pIGwJ3dgrvKiW7Ig9X826w47mXpqpPlSySjeO2ErZyKKOlBnFo5Pxxiip3g8CNaNs9Zj4oDijafTqRlbkSiGzZWgFGYr6N66vHqpVtFHsqlU6ZQebRdgmlC0M2IeMPYdBOSpU7UlBHbHVTVvBnpcC6XpcOjpGo1eTXZ2Zn24YFWz9Hfx0EwzXOw0DOQ5elcB5REr6j0xgIv0Bwor2fWg3DYTBR1pPVOdfCXGagPiWt4fv8e11yEaPe45opEu8c1Vy7iDWPzHtdcUlzjWy5yefqrEde8WIZuFBmpf6tnjC8vrmnePRksrqlHMdYZoph9i%2B6XuGjqLsG2bf0mPQI4Sqozn3QcZPmtHsF0LfTiD%2BMRgG60lxv6PML4ZA4BHFQ4CCNeDe6uGnD7iPuEmmyXdMlv8nxmn9tZKOgtPLWXJcDfyxIDVhf2Lmt1IeE89YXGXZmj6wv1s3md0YnrC6B5ZeOOzz0SF%2F34VJUPxCc0qrAC9Ss%2FR%2BOz7ilPjc%2FDrogMhk8BM%2B0QmJ2h2t%2BLT3tPeNo3gc5GZtOEA6GTNYtb7Fn34l8B4Kc%2F

10.
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7Vtdc7I6EP41XNaBRBAvW32%2FLs7UM704H3cpBM0UwRNC1f76k0DCNy9o0apjpzMlS7Jhd59kN7upBmfr3Q%2BKNqs%2FQhf7GtDdnQbnGgA2AJr41d19ShjrekpYUuKmJCMnvJAPLImqW0xcHJU6sjD0GdmUiU4YBNhhJRqiNNyWu3mhX551g5a4RnhxkF%2Bn%2FkVctlJiTXL6T0yWKzWzYU3TN2ukOktJohVyw22BBL9pcEbDkKVP690M%2B0J3Si%2FpuO8tb7MPozhgfQZEbmya%2F5KPVfzjxfhp%2FvmxDaIHyeUd%2BbEUmOGIjdiOacDyOd%2BnV8qfluJJUVzyLsRhe6kj679YyPDE8I49IJ8sAw0%2B8h4%2B9uQw2UExGo1Gihf%2F3ITdJ2fgdg3YQ5SgR7ww9M2ucWo1g%2BifoMYPaWkKDUDPg%2FynSGr4lL4T8gWgc0ToaTd7rBckTz%2Bi%2FGHXoZBm2YzzyNYmQhujIVWh8x%2FPa1aFXVAFGFQVh0rQwHoIoeDxQpVJ9T2leegh1MFg1P1xoDQR4Bv7RjwGIeN%2FnrYrwvDLBjmCtuVOkdNWbO3zliHYI%2BdtScM4cJ9j5pMAS7qL6NszH0WY8JT6SDc5MbX53LCyed8x5RK1ugAjcyzcIeNwjRnd8y475XzTEXvlrMDITCnb3LcZyvWuin4Nyo5I%2BtNlxjx3OfxBep0DPBCoeaAFDR0cRTim7SY5D6Vi6MRq2JUG67DzAOYCesVeKpQoWCvrU7KW6ji4tWDNWs%2FU5aEXCpwLsNeJ7K4W%2BCYF5vFrXC5nvrghAKatW2PTsAww1eFA67uywO366oaTJrxMT4WXcQ0vNe3iwH0UgTpvOT6KIuKUlZmvOu6InvCOsL8Lz%2F8k%2BjRla74rvJrvVSPgoiSDRlPbVoTCSNHMhyat0tgFpoTrA1NJTGXAbu3oUDEUlzOMqYN%2FoyCpD4boErOuQL1u%2BIJlzQbDKhrFPmLkvfy5TcaWMyxCkoQLEldwWtmIrApeUjHlqOIZpMKoCtDM1yhGqR5qjBLsZWIfD0dzcDhm0CrDCnThqj%2BEOqEBvxIa46qPMq8UGtZ5dqpJz63KKONpMhyeOrck2BN34L4lDYC7aQPuSke2zx8H89Bm8Wuen%2BfSJNvJsjyKUbRBQSOnPEp6kMdSwY8EhBHkH35ES%2Bc561FxQNEWsydlmRuR6IaN5RMcZOuomjC9eunWoYsz6bQZ1B6nDiUspvhmRLxhbDoxTVMnesKIewddt25GOpLJVU5H9peq0eTlbGZr2kcEWh1Hf4%2F4%2FixLskLXxLY7FjpgNHzDhTc2eIXWQHk9qxqEw3qiqCWtd6qDv8pAfUlcI6oS97jmIkS7xzVXJNo9rrlyEW8Ym%2Fe45pLiGs92sCPSX7W45tU2x2aekfpcPWNyeXFN%2FcbMYHFNNYqxzxDF9C3FX%2BKiuW8Jl7QluAjbXuOWYDk2fvWG2RLA2GyuN3RtCZOT7QjgoMpBEIpycHvZQNhHXoM0VLugS3FX5zv%2F3NZKQWflqbkuAX5flxiwvNC7rtWGhPMUGGqXZY4uMFQP51VGJy4wgPqdjTs%2Be2QuuvGpa1%2BIT2iWYQWqd36Oxmd1pzw1Pg%2B7IzIYPiXMjENgdoZyfyc%2Bpz3hOb0JdNZSmxYcCJ28mV%2B%2BT7vn%2F8EAv%2F0P

11.
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7Vtdc7I6EP41XNaBRBAuq75fF2fqmV68H3cUgmaK4Amhan%2F9SSDhmxe0aNWx05mSJdmwu0%2Bym91UgbP17huxN6t%2FQhf5ClDdnQLnCgAmAAr%2FVd19ShirakpYEuymJC0nPON3JIiyW4xdFJU60jD0Kd6UiU4YBMihJZpNSLgtd%2FNCvzzrxl6iGuHZsf069Sd26UqKNcnp3xFeruTMmmGlb9a27CwkiVa2G24LJPhFgTMShjR9Wu9myOe6k3pJx31teZt9GEEB7TMgcmNd%2F4PfV%2FG3Z%2B27%2Fu%2F7NogeBJc324%2BFwBRFdER3VAGGz%2FhOXwh7WvInSXHxGxeH7oWOjP9iLsOUoh19sH28DBT4yHr4yBPDRAfJaDQaSV7scxN2H5yB2TWgD1GCHv5CUze7xqnlDLx%2Fgho%2FJKUpFAA9D7KfIqnhU%2FpOyBaAyhChpt3MsVqQPP2I8oddh0KaZdPOI1ubCG2MhlSFyn48r1kVZkEVYFBVHCpBA%2Bsh7AuPF6pMqu8pzUMPoQ4Go%2B6PA6WJANvYN%2FwxCCn7M92uMEXPG9vhtC1zioy2omuftTTO3nZelySMA%2Fcppj4OkKC7Nnl9YqMw5Z5SHak6I6Y2n2tGNu8bIkyiVhegZY6FOWQUrhEle9ZlJ51vOmIvnRUY6Sllm%2Fs2TbreVdGvQdHRFv50mTHPXQ57EF7nAA8Eah5oQUIHRRGKSbtJzkOpGDqxGnKFwTrsPIC5gFqxlwwlCtbK%2BpSsJTsObi1Ys9YTcVnoZQfOBdjrRHaXC3yTAvP4NS6WM1vcEADdVI2xrhkasFQ40PquLHCzvrrhpAkv1qnwMq7hpaZdFLiPPFBnLce3owg7ZWXmq445oinaYfqr8Pw70acuWvNd4dV8LxsBEyUZNLJMUxIKI3kzH5q0SmMXiGCmD0QEMZUBubWjQ8VQTM4wJg76i4KEPqhNloh2Bep1wxcsqzcYVtII8m2K38qf22RsMcMixEm4IHAFrcpGZFTwkoopRhXPIBVGVYBmvkYySvVQY5RgLxP7eDjqg8Mxg1YZVqALV%2F0h1AkN%2BJnQGFd9lH6l0DDOs1NNem5VWhlPk%2BHw1LklwZ64A%2FctaQDcWQ24Kx3ZPn4czEObxY95fp5Lk2wny%2FJIRtHGDho55VHSgziWcn44wBTb%2FuFHtHSesx4VBxRtMZtKy9yIRDdsLB%2BjIFtH1YTp1Uu3Dl2USafMoPJoOQTTmKCbEfGGsenEJE2dqAkj5h1U1bgZ6XAmVzkd2V%2BqRpOXs5mtaR8eaHUc%2FT3s%2B7MsyQpdHZnumOuAkvAVFd6Y4AUaA%2BX1jGoQDuuJopa03qkO%2FjID9SlxDU%2FF3%2BOaixDtHtdckWj3uObKRbxhbDbENZp5M9IV45rxNcQ1NjI9nv6qxTWGY6IXL5vvY%2FWMyeXFNfUbM4PFNdUoxjpDFNO3FH%2BJi%2BZ%2B1LmkLcEzHeQ0bgkvpj7W8yT1x2riY7253tC1JUxOtiOAgyoHQcjLwe1lA24fcQ1Sk%2B2CLvkFpK%2Fsc1srBZ2Vp%2Ba6BPh7XWLA8kLvulYbEs5TYKhdljm6wFA9nFcZnbjAAOp3Nu747JG56ManqnwiPqFehhWo3vk5Gp%2FVnfLU%2BDzsjshg%2BBQw0w6B2RnK%2FZ34tHrC07oJdNZSmwYcCJ2smV%2B%2BT7vn%2F8EAv%2FwP

12.
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&title=Processus#R7Vtdc6o8EP41XNaBBBAuq56vizP1nV68H3cUgmaK4Amhan%2F9m0DCN0UtWnXsdKZkSTbs7pPdTTZV4HS1%2FUGc9fJ35KFAAaq3VeBMAcACQOG%2FqrfLCLqqZoQFwV5G0grCM35Hgii7JdhDcaUjjaKA4nWV6EZhiFxaoTmERJtqNz8KqrOunQVqEJ5dJ2hS%2F8YeXUqxxgX9J8KLpZxZM%2B3szcqRnYUk8dLxok2JBL8pcEqiiGZPq%2B0UBVx3Ui%2FZuO8db%2FMPIyik%2BwyIvcQw%2FsPvy%2BTHs%2FbT%2BOt9E8YPgsubEyRCYIpiOqJbqgAzYHwnL4Q9LfiTpHj4jYtDd0JH5p%2BEyzChaEsfnAAvQgU%2Bsh4B8sUw0UEyGo1Gkhf73JTdJ2dgdg3pQ5yih7%2FQ1PW2dWo5A%2B%2BfoiaISGUKBUDfh%2BynTGr5lH0nZAtAZYhQs26WrpYkzz6i%2BmHXoZB22bTzyNYlQhejIVWhsh%2Ffb1eFVVIFGFQVh0rQwvpjoWybu%2BUWobgbzoWCxwtVJTV9SvvQQ6iDwaj%2F40BlIsAc%2B5o%2FhhFlfyabJaboee24nLZhQZHRlnQVsJbG2Tvu64JESeg9JTTAIRJ0zyGvT2wUpjxSqiPVYMTM5jPNzOd9Q4RJ1BkCtDywsICMohWiZMe6bGXwzUbsZLACIyOjbIrYpsnQuyzHNSg6OiKeLnLmRchhDyLqHBCBQCMCzUnkojhGCek2yXkoNUOnVkOeMFiPnQcwF1Br9pKpRMlaeZ%2BKtWTHwa0FG9Z6Ih5LvZzQvQB7ncjucoGvM2Aev8bFcmaLGwJgWKqpG5qpAVuFA63v2gK3mqsbjtvwYp8KL3oDLw3totB75Ik6a7mBE8fYrSqzWHUsEE3QFtN%2FSs%2F%2Fpvo0RGu2Lb2a7WQjZKKkg0a2ZUlCaSRvFkPTVmXsHBHM9IGIIGYyIK%2BxdagZiskZJcRFHyhI6IM6ZIFoX6LeNHzJskaLYSWNoMCh%2BK36uW3GFjPMI5ymCwJX0K45IrOGl0xMMaq8B6kxqgM0jzWSUaaHBqMUe7nYx8PRGByOObSqsAJ9uNofQr3QgF8JDb0eo4wrhYZ5Hk813tNVaVU8jYfDU69LgnviDtxd0gC4s1twV9myfX47WKQ281%2BzYj%2BX7d5PdsojGcVrJ2zlVGRJD2JbyvnhEFPsBIdv0bJ5zrpVHFC0%2BXQiLXMjEt2wsQKMwnwd1Q9Mr166VeShXDplCpVH2yWYJgTdjIg3jE03IdnRiZoyYtFB1aybkQ7nclm6fpRUrSavnmZ2HvvwRKtn6%2B%2FjIJjmh6zQt1zk8jQxpiR6RaU3L5ahG0Xm9ql9v1lPwmHzoKjjWO9UG395AvU1eY3Vbe1LRPUNu6N7XnNFot3zmisX8Yaxec9rLimv8Rxk%2Ba15jela6MXP5%2FtcPWN8eXlN88bMYHlNLYvR1G7bnrHafbmLpsUlqOZNugR4lFRndgkGsjy9zSVY4AWaA11hALrRXm%2Focwnjk3kEcFDlIIx4Obi7bMDtI65BarJd0iW%2FgPSdfW5npaC38tRelwAf1yUGLC%2FsXdfqQsJ5CgyNyzJHFxjqm%2FM6oxMXGEDzzsYdn3ucXPTjU1W%2BEJ%2FQqMIK1O%2F8HI3Puqc8NT4PuyMyGD4FzLRDYHaGcn8vPu094WnfBDobR5smHAidrFlcvs%2B6F%2F%2FBAL%2F9Dw%3D%3D
-->

### Interblocage
Un tel accès concurrent à une même ressource est souvent problématique.  
Certains périphériques matériels, en particulier, nécessitent un **accès exclusif**.  
Nous illustrons ce phénomène complexe avec les deux programmes suivants :
* **`enregistrer_micro`** : acquiert la carte son en accès exclusif (pour accéder au micro) et écrit les sons enregistrés sous un certain format (par exemple `mp3` ou `wav`) sur sa sortie standard et s'arrête lorsqu'il a écrit l'équivalent de 10 secondes de son.
* **`jouer_son`** : acquiert la carte son en accès exclusif (pour accéder au haut-parleur) et joue le contenu qu'il reçoit sur son entrée standard.

Même si nous ne donnons pas les détails du code (d'assez bas niveau) permettant de réaliser ces programmes, ils n'en restent pas moins assez réalistes.  
Les **cartes son** sont typiquement des périphériques ne pouvant être utilisés que par, au plus, un processus (dans les systèmes d'exploitation actuels, un processus particulier de type daemon agit comme un *serveur de sons* : il est le seul à accéder directement à la carte son, les autres processus communiquent avec lui pour lui demander d'enregistrer ou jouer des sons).  

Ces deux programmes peuvent être exécutés séquentiellement de la façon suivante :
```
utilisateur@machine:~$ enregistrer_micro > message.mp3
utilisateur@machine:~$ cat message.mp3 | jouer_son
```
* La première ligne de commande appelle le programme `enregistrer_micro` et enregistre (au moyen d'une redirection) les octets émis sur sa sortie standard dans le fichier `message.mp3`.  
* La deuxième commande utilise l'utilitaire `cat` pour afficher le contenu du fichier `message.mp3` sur la sortie standard puis redirige cette dernière au moyen de l'opérateur `|` sur l'entrée standard de la commande `jouer_son`.  

Que se passe-t-il maintenant si on enchaîne directement les deux commandes?

```
utilisateur@machine:~$ enregistrer_micro | jouer_son
```

* Le premier programme ouvre la carte son.  
Il commence ensuite à envoyer des données sur sa sortie standard.  
* Le second programme tente alors d'ouvrir la carte son et se retrouve bloqué.  
Il ne peut continuer son exécution tant que la carte son n'est pas disponible.  

Comme le programme `jouer_son` est bloqué, il ne lit aucun octet sur son entrée standard. Comme il ne lit aucun octet, le premier programme se retrouve lui aussi bloqué lorsqu'il tente d'écrire sur sa sortie standard (puisque personne ne lit, les octets s'accumulent dans une zone mémoire tampon et le programme est bloqué quand cette dernière est pleine).  
Les deux processus correspondant à ces deux programmes sont en **interblocage** :
* le processus de `jouer_son` attend que la carte son soit libre pour progresser et lire son entrée standard
* le processus d'`enregistrer_micro` attend que quelqu'un consomme son entrée standard pour progresser et libérer la carte son

L'interblocage est le grand danger de la programmation concurrente.  
Dans notre exemple, l'utilisateur constaterait juste que les deux programmes ne font rien et ne rendent pas la main.  

Il existe quatre conditions nécessaires à la présence d'un interblocage, appelées **conditions de Coffman**, du nom d'Edward Grady Coffman Jr. (1934-), informaticien américain qui les a décrites en premier en 1971.
1. **Exclusion mutuelle** : au moins une ressource du système doit être en accès exclusif.
2. **Rétention et attente** : un processus détient une ressource et demande une autre ressource détenue par un autre processus.
3. **Non préemption** : une ressource ne peut être rendue que par un processus qui la détient (et ne peut pas être *préemptée* ou acquise de force par un autre processus).
4. **Attente circulaire** : l'ensemble des processus bloqués $P_1$, ... , $P_n$ sont tels que $P_1$ attend une ressource tenue par $P_2$, ... , $P_n$ attend une ressource détenue par $P_1$.

Dans notre exemple précédent, les quatre conditions sont remplies.  
* Il existe une ressource à accès exclusif, la carte son (1).  
* Le processus du programme `enregistrer_micro` détient la carte son et veut, en plus, pouvoir écrire sur sa sortie (2).  
* Nous avons fait l'hypothèse que `jouer_son` est bloqué et ne peut pas acquérir la carte son (3).  
* Le programme `enregistrer_micro` attend le déblocage de son entrée standard bloquée par `jouer_son` et ce dernier attend le déblocage de la carte son détenue par `enregistrer_micro` (4).

Il existe plusieurs stratégies permettant d'éviter les interblocages ou de les détecter et de les résoudre.  
Dans le cadre générique des systèmes d'exploitation, et des programmes utilisateurs, la solution souvent retenue est la plus simple, à savoir interrompre les programmes (par exemple au moyen de la commande [`kill`](https://pubs.opengroup.org/onlinepubs/009604499/utilities/kill.html)).

### Programmation concurrente en Python
Afin d'illustrer les problématiques d'interblocage dans un cadre plus contrôlé que dans un système d'exploitation, nous donnons ici une introduction à la **programmation multithread** en Python.  
Comme nous l'avons déjà expliqué, un thread est un *sous-processus* démarré par un processus et s'exécutant de manière concurrente avec le reste du programme.  
Le module [`threading`](https://docs.python.org/fr/3/library/threading.html) de la bibliothèque standard Python permet de démarrer des **threads**.  
Nous illustrons son utilisation au moyen du programme [`pr_threads.py`](Fichiers/pr_threads.py) suivant :

In [ ]:
import threading

def hello(n):
    for i in range(5):
        print(f"Je suis le thread {n} et ma valeur est {i}")
    print(f"------ Fin du Thread {n}")

for n in range(4):
    t = threading.Thread(target=hello, args=[n])
    t.start()

Ce programme définit une fonction `hello` prenant en argument un entier représentant l'identifiant du thread dans lequel on se trouve.  
Cette fonction effectue ensuite une boucle, pour `i` allant de `0` à `4`, et écrit, à chaque tour de boucle, la valeur de `n` et de `i`.  
La fonction imprime ensuite un message indiquant qu'elle a terminé la boucle puis se termine.  

Le programme principal effectue une boucle et appelle quatre fois (pour `n` entre `0` et `3`) l'expression `threading.Thread(target=hello, args=[n])`.  
Cette dernière crée un objet de type [`Thread`](https://docs.python.org/fr/3/library/threading.html#threading.Thread).  
L'argument nommé `target` doit être une fonction et l'argument `args` un tableau des arguments qui seront passés à la fonction.  
La variable `t` contient l'objet `Thread` créé.  
La méthode [`.start()`](https://docs.python.org/fr/3/library/threading.html#threading.Thread.start) lance l'exécution de la fonction en tâche de fond.  
Cette méthode rend directement la main et le programme principal continue de s'exécuter de façon concurrente au thread démarré.  

Le programme ci-dessus, une fois exécuté, comporte alors cinq threads : ceux démarrés par `.start()` et le thread principal.  
Voici un affichage possible pour ce programme :

    Je suis le thread 0 et ma valeur est 0
    Je suis le thread 0 et ma valeur est 1
    Je suis le thread 0 et ma valeur est 2
    Je suis le thread 1 et ma valeur est 0
    Je suis le thread 1 et ma valeur est 1
    Je suis le thread 1 et ma valeur est 2
    Je suis le thread 1 et ma valeur est 3
    Je suis le thread 2 et ma valeur est 0
    Je suis le thread 2 et ma valeur est 1
    Je suis le thread 2 et ma valeur est 2
    Je suis le thread 2 et ma valeur est 3
    Je suis le thread 2 et ma valeur est 4
    ------ Fin du Thread 2
    Je suis le thread 1 et ma valeur est 4
    ------ Fin du Thread 1
    Je suis le thread 3 et ma valeur est 0
    Je suis le thread 3 et ma valeur est 1
    Je suis le thread 3 et ma valeur est 2
    Je suis le thread 3 et ma valeur est 3
    Je suis le thread 3 et ma valeur est 4
    ------ Fin du Thread 3
    Je suis le thread 0 et ma valeur est 3
    Je suis le thread 0 et ma valeur est 4
    ------ Fin du Thread 0
    
Comme pour les processus, les threads alternent leur exécution au gré des commutations de contexte.  
Deux exécutions successives donnent des affichages différents.  
L'ordre dans lequel sont démarrés les threads ne donne aucune indication sur l'ordre dans lequel ils peuvent se terminer (dans l'exemple ci-dessus, le thread 0, démarré en premier se termine le dernier).  

On peut trouver étonnant que les threads ne s'interrompent pas les un les autres au milieu d'une ligne (ce phénomène s'explique par la notion de **verrous cachés**).

Les threads peuvent servir à illustrer les problèmes de concurrence et d'interblocage.  
Considérons le programme [`pb_threads.py`](Fichiers/pb_threads.py) suivant :

In [ ]:
import threading

compteur = 0

def incrc():
    global compteur
    for _ in range(100_000):
        v = compteur
        compteur = v + 1

th = []
for n in range(4):
    t = threading.Thread(target=incrc, args=[])
    t.start()
    th.append(t)

# On s'assure de la terminaison de l'exécution des threads
for t in th:
    t.join()
    
print(f"Valeur finale : {compteur}")

Ce dernier définit une variable globale `compteur`.  

La fonction `incrc` est similaire à la fonction `hello` du programme précédent.  
Elle ne prend pas d'argument mais exécute `100000` itérations d'une boucle qui incrémente la variable globale `compteur`.  

Le programme principal déclare un tableau vide `th`.  
Il démarre ensuite quatre threads et stocke les objets correspondants dans le tableau `th`, après les avoir démarrés.  

Enfin, pour chacun des objets `Thread` stockés, la méthode [`.join()`](https://docs.python.org/fr/3/library/threading.html#threading.Thread.join) est appelée.  
Cette dernière permet d'attendre que le thread auquel on l'applique soit terminé.  
Si le thread est déjà terminé, la méthode se termine immédiatement.  

Enfin, le programme imprime la valeur finale contenue dans le compteur.  

Comme on a démarré quatre threads, et que chacun incrémente la valeur `100000` fois, on s'attend à ce que l'affichage final soit `400000`.  
Cependant, si on exécute le programme plusieurs fois, on peut constater qu'il n'affiche pas toujours le nombre attendu :

```
utilisateur@machine:~$ python3 pb_threads.py
Valeur finale : 344447
```

Que se passe-t-il?  
Considérons les quatre threads $t_0$ à $t_3$.  
Supposons que $t_0$ soit en exécution et que la valeur de `compteur` soit `42`.  
* Si $t_0$ est interrompu juste après avoir exécuté `v = compteur`, alors sa variable locale `v` contient la valeur `42`.  
* La commutation de contexte donne la main à $t_1$, qui exécute `v = compteur` suivi de `compteur = v + 1` avant d'être lui-même interrompu.  
* La valeur de compteur continue d'augmenter lors des commutations de contexte suivantes avec $t_2$ et $t_3$ jusqu'à avoir `compteur` valant `50`.  
* Lorsque $t_0$ reprend enfin la main, il continue là où il s'était arrêté et exécute donc `compteur = v + 1`, où la valeur de `v` est `42`.  

Le thread $t_0$ va donc écraser la valeur `50` avec la valeur `43`. 

Pour corriger ce problème, il nous faut donc garantir l'**accès exclusif** à la variable `compteur` entre sa lecture et son écriture.  
On peut, pour cela, utiliser un **verrou**.  
Un verrou est un objet que l'on peut essayer d'acquérir. 
* Si on est le premier à faire cette demande, on acquiert le verrou.  
On peut le rendre à tout moment. 
* Si en revanche quelqu'un d'autre tient le verrou, alors on est bloqué jusqu'à ce qu'il soit libéré. 

Des verrous munis de ces deux opérations sont disponibles dans le module `threading` avec le constructeur [`Lock`](https://docs.python.org/fr/3/library/threading.html#threading.Lock).  
Une fois l'objet verrou construit, on peut tenter de l'acquérir avec la méthode [`.acquire()`](https://docs.python.org/fr/3/library/threading.html#threading.Lock.acquire) et on peut le rendre avec la méthode [`.release()`](https://docs.python.org/fr/3/library/threading.html#threading.Lock.release).  
Une manière de corriger le programme [`pb_threads.py`](Fichiers/pb_threads.py) est la suivante :

In [ ]:
import threading

verrou = threading.Lock()
compteur = 0

def incrc():
    global compteur
    for _ in range(100_000):
        verrou.acquire()
        v = compteur
        compteur = v + 1
        verrou.release()

th = []
for n in range(4):
    t = threading.Thread(target=incrc, args=[])
    t.start()
    th.append(t)

for t in th:
    t.join()
    
print(f"Valeur finale : {compteur}")

Avant toute tentative de lecture, on essaye d'acquérir le verrou.  
Une fois ce dernier acquis, le thread courant a la garantie qu'il est le seul à exécuter son code, jusqu'à l'instruction `verrou.release()`.  
Une telle portion de code protégée par un verrou s'appelle une **section critique**.  

Attention, cela ne signifie pas que le thread ne peut pas être interrompu entre les lignes `v = compteur` et `compteur = v + 1`.  
Cela signifie seulement que les autres threads, s'il reprennent la main, ne sont pas eux-mêmes en section critique (ils sont forcément ailleurs dans leur code, probablement bloqués sur l'instruction `verrou.acquire()`).  

On remarque qu'il est important que tous les threads manipulent le même verrou. C'est pour cela qu'il a été défini dans une variable globale accessible depuis tous les threads.  

En dernière remarque, on peut ajouter que le programme aurait été tout aussi faux si on avait écrit `compteur = compteur + 1` ou même `compteur += 1`.  
En effet, une instruction d'un langage de haut niveau comme Python est en fait décomposée en de nombreuses instructions machines : 
* lecture de la valeur du compteur en mémoire
* addition 
* écriture en mémoire de la nouvelle valeur  

In [ ]:
import dis

code = """
v = compteur
compteur = v + 1
"""

dis.dis(code)

In [ ]:
import dis

dis.dis("compteur = compteur + 1")

In [ ]:
import dis

dis.dis("compteur += 1")

Le programme aurait donc pu être interrompu entre deux de ces instructions machines.

L'utilisation de plusieurs verrous rend les interblocages possibles.  
Il conviendra donc d'être très prudent lorsque l'on manipule deux verrous à la fois.  
On illustre ce problème avec le programme suivant :

In [ ]:
import threading

verrou1 = threading.Lock()
verrou2 = threading.Lock()

def f1():
    verrou1.acquire()
    print("Section critique 1.1")
    verrou2.acquire()
    print("Section critique 1.2")
    verrou2.release()
    verrou1.release()

def f2():
    verrou2.acquire()
    print("Section critique 2.1")
    verrou1.acquire()
    print("Section critique 2.2")
    verrou1.release()
    verrou2.release()

t1 = threading.Thread(target=f1, args=[])
t2 = threading.Thread(target=f2, args=[])
t1.start()
t2.start()

Ce dernier déclare deux verrous, utilisés de façon symétrique par deux fonctions `f1` et `f2`.  
La fonction `f1` essaye d'acquérir d'abord `verrou1` puis `verrou2`, alors que `f2` essaye de les acquérir dans l'ordre inverse.  
Si on exécute ce programme, il a de grandes chances de se retrouver bloqué.  

Considérons l'exécution suivante :
* Le thread `t1` a la main.  
Il s'exécute jusqu'à son premier affichage (avant la tentative d'acquisition de `verrou2`).
* Le thread `t2` prend la main.  
Il s'exécute, acquiert `verrou2` qui est toujours libre, puis bloque sur l'acquisition de `verrou1`.
* Le thread `t1` reprend la main, il bloque alors sur l'acquisition de `verrou2` (tenu par `t2`).

Chaque thread détient un verrou et attend l'autre. Ils sont en interblocage.  

Cependant, le problème ne se manifeste que si les exécutions se font dans cet ordre.  
Si la commutation de contexte intervient après que `f1` a acquis `verrou2`, alors `t1` peut se terminer sans bloquer.  

Dans des programmes complexes, les situations d'interblocage sont particulièrement difficiles à tester et à corriger.  
En effet, à cause du non déterminisme de l'ordonnancement des threads et des processus, il se peut que le programme se comporte bien lors de la phase de test et ne se bloque que lorsqu'il est exécuté en conditions réelles.

#### Verrous cachés
Considérons de nouveau le programme : 

In [ ]:
import threading

def hello(n):
    for i in range(5):
        print(f"Je suis le thread {n} et ma valeur est {i}")
    print(f"------ Fin du Thread {n}")

for n in range(4):
    t = threading.Thread(target=hello, args=[n])
    t.start()

Puisque les commutations de contextes peuvent intervenir à tout moment, il peut sembler étrange que les affichages des threads s'entrelacent *proprement*.  
En effet, l'affichage du programme donne l'impression que les threads ne sont jamais interrompus au milieu d'une ligne.  
Ce comportement est du au fait que, par défaut, la sortie standard de Python possède un **buffer** (ou zone tampon). Ce dernier n'est rien d'autre qu'un tableau d'octets.  
Chaque appel à `print` n'écrit pas directement dans la console, ce qui serait inefficace, mais ajoute les caractères à la suite dans le buffer.  
Lorsque ce dernier est plein, il est affiché d'un seul coup en utilisant une implémentation bas niveau du système d'exploitation.  
Pour les fichiers textes (comme la sortie standard) le buffer est vidé à chaque retour chariot.  
Hors, ce buffer n'est rien d'autre qu'un tableau d'octets, accompagnés d'entiers représentant sa taille, et la position du dernier caractère écrit.

Pour éviter le phénomène illustré dans le programme suivant :

In [ ]:
import threading

compteur = 0

def incrc():
    global compteur
    for _ in range(100000):
        v = compteur
        compteur = v + 1

th = []
for n in range(4):
    t = threading.Thread(target=incrc, args=[])
    t.start()
    th.append(t)

for t in th:
    t.join()
    
print(f"Valeur finale : {compteur}")

Les mises à jour du buffer ainsi que les modifications de ces entiers sont **protégées par un verrou**.  
En effet, il est naturel de vouloir utiliser `print` depuis plusieurs threads différents.  
Si on écrit une chaîne se terminant avec retour chariot (comportement par défaut de `print`) il se produit donc le phénomène suivant :
* acquisition d'un verrou associé au buffer
* écriture de tous les caractères dans le buffer
* arrivé au retour chariot, envoyer lecontenu du buffer dans la console, puis le vider
* relâcher le verrou.

Deux threads ne peuvent donc pas mélanger leur ligne, puisque l'écriture d'une ligne devient une section critique.  

On peut obtenir un comportement plus *réaliste* en désactivant les buffers d'écriture globalement, avec l'option `-u` de l'interprète Python:

```
utilisateur@machine:~$ python3 -u pr_thread.py
Je suis le thread 0 Je suis le thread 1Je
suis le thread 2 et ma valeur estet
ma valeur est et ma valeur estO
Je suis le thread 00
et ma valeur estJe suis le thread 1 20
et ma valeur estJe suis le thread 11
Je suis le threadet ma valeur est 21
```

## Exercices 
### Exercice 1
On suppose qu'Alice exécute dans son terminal la commande `ps -a -u -x`.  
Le processus correspondant à cette commande fera partie des processus affichés dans la sortie.  

Dire quel sera l'état de ce processus (R ou S) et justifier.

### Exercice 2
On considère le programme suivant.

In [ ]:
import threading

compteur = 0

def incrc():
    global compteur
    for _ in range(100000):
        v = compteur
        compteur = v + 1

th = []
for n in range(4):
    t = threading.Thread(target=incrc, args=[])
    t.start()
    th.append(t)

for t in th:
    t.join()
print(f"Valeur finale : {compteur}")

Pour chacune des affirmations suivantes, dire si elle est vraie ou fausse et justifier.
1. Le programme affiche toujours 400000.
2. Le programme peut afficher un nombre plus petit que 400000.
3. Le programme peut afficher un nombre plus grand que 400000.
4. Si on ajoute `t.join()` après `t.start()`, le programme affiche toujours 400000.
5. Si on transforme le corps de la boucle de la fonction `incrc()` par :  
```python
verrou.acquire()
v = compteur
verrou.release()
verrou.acquire()
compteur = v + 1
verrou.release()
```
en ayant définie un variable globale `verrou = threading.Lock()`, alors le programme affiche toujours 400000.

### Exercice 3
On considère la situation de la figure suivante :

<div style="text-align: center">
   <img alt="Exercice" src="Images/blocage-1.png" > 
</div>

dans laquelle quatre voitures sont bloquées à une intersection.  

Montrer qu'il s'agit d'un interblocage, c'est-à-dire que les quatre conditions de Coffman sont réunies.  
On indiquera précisément quelles sont les ressources et les processus dans cette situation.  
On fera l'hypothèse que les conducteurs sont raisonnables, i.e qu'ils ne veulent pas provoquer d'accident.

### Exercice 4
Considérons un petit système embarqué : un petit ordinateur relié à trois LED A, B et C.  
Une LED peut être éteinte ou allumée et on peut configurer sa couleur.  
On dispose de trois programmes qui affichent des signaux lumineux en faisant clignoter les LED.

Chaque programme possède une LED primaire et une LED secondaire. 

* Le programme $P_1$ affiche ses signaux sur A (primaire) et B (secondaire) en vert.  
* Le programme $P_2$ affiche ses signaux sur B (primaire) et C (secondaire) en orange.  
* Le programme $P_3$ affiche ses signaux sur C (primaire) et A (secondaire) en rouge.

Comme les LED ne supportent pas d'être configurées dans deux couleurs en même temps, le système propose deux primitives `acquerirLED(nom)` et `rendreLED(nom)` qui permettent respectivement d'acquérir et de relâcher une LED.  
Si une LED est déjà acquise, alors `acquerirLED()` bloque.

On suppose que chacun des trois programmes $P_1$, $P_2$ et $P_3$ effectue les actions suivantes en boucle:
1. acquérir sa LED primaire
2. acquérir sa LED secondaire
3. configurer les couleurs
4. émettre des signaux
5. rendre la LED secondaire
6. rendre la LED primaire
7. recommencer en 1

Montrer qu'il existe un entrelacement des exécutions qui place $P_1$, $P_2$ et $P_3$ en interblocage.

### Exercice 5
Écrire un programme Python simulant le code de l'exercice précédent.  
On pourra s'inspirer du programme [`interblocage.py`](Fichiers/interblocage.py).  
Constater qu'en exécutant suffisamment de fois votre programme il se bloque.

Indication : afin de laisser plus de chance au système de changer de contexte, on pourra mettre des affichages (comme dans le programme [`interblocage.py`](Fichiers/interblocage.py)) juste après l'acquisition d'un verrou. En effet, l'écriture dans la console passe le thread courant en attente, le temps que les écritures soient effectuées, ce qui laisse une opportunité à l'ordonnanceur de choisir un autre thread ou un autre processus.

### Exercice 6
Si deux transactions travaillent sur un même objet (par exemple sur une même table), alors la seconde est bloquée jusqu'à ce que la première soit terminée.

Nous pouvons maintenant expliciter ce mécanisme : lors qu'une transaction accède à une table, elle tente de prendre un verrou sur cette dernière.

Les verrous sont relâchés au moment du `COMMIT` ou `ROLLBACK`.

La réalité est plus complexe. Pour des raisons de performances, les SGBD tentent d'acquérir les verrous le plus tard possible et de les relâcher le plus tôt possible, mais nous n'entrons pas dans ces détails. 

On considère deux tables :

```sql
CREATE TABLE T (num INTEGER);
CREATE TABLE S (num INTEGER);
INSERT INTO T VALUES (1000);
INSERT INTO S VALUES (1000);
```

qui peuvent représenter de façon simplifiée des comptes en banque.  

Considérons les deux transactions suivantes :

```sql
START TRANSACTION;
    UPDATE T SET num = num + 100;
    UPDATE S SET num = num - 100;
COMMIT;
```

et

```sql
START TRANSACTION;
    UPDATE S SET num = num + 100;
    UPDATE T SET num = num - 100;
COMMIT;
```

La première simule un virement de `S` vers `T` et la seconde un virement de `T` vers `S`.  

Montrer qu'il s'agit d'un interblocage, c'est-à-dire que les quatre conditions de Coffman sont réunies.

### Exercice 7
L'ordonnancement d'un ensemble de processus produit une séquence qui indique quel est le processus élu à chaque pas de temps.  
Le but de cet exercice est d'écrire des programmes Python pour produire une telle séquence à partir d'ensemble de processus donnés, et pour différents algorithmes d'ordonnancement.

On utilise un dictionnaire pour définir les caractéristiques d'un processus : date de création, durée, éventuellement priorité si cela est pertinent pour l'algorithme d'ordonnancement, etc.

On écrira, par exemple :

In [ ]:
data = [
    {'nom': 'PA', 't0': 0, 'duree': 10},
    {'nom': 'PB', 't0': 3, 'duree': 4},
    {'nom': 'PC', 't0': 2, 'duree': 8},
]

Ces données peuvent être codées, en dur, dans un programme ou lues depuis un fichier CSV.

#### Ordonnancement FIFO
1 . Implémenter un ordonnancement FIFO qui produira, sur les données fournies en exemple, un résultat de la forme suivante : 

```python
{'PA', 'PA', 'PA', 'PA', 'PA', 'PA', 'PA', 'PA', 
 'PA', 'PA', 'PC', 'PC', 'PC', 'PC', 'PC', 'PC', 
 'PC', 'PC', 'PB', 'PB', 'PB', 'PB'}
```

On simulera l'évolution du temps.  
A partir des données d'entrée, on pourra nourrir et gérer une structure de données appropriée pour mémoriser les processus.  
A chaque processus, sera attachée la durées d'exécution restante.

2 . Compléter l'implémentation par une fonction permettant d'afficher le ŕesultat sous une forme plus lisible.  
Par exemple : 
```python
"PA prend la main en t=0."
"PC prend la main en t=12."
"PB prend la main en t=22."
"Fin d'exécution en t=26"
```

#### Ordonnancement tourniquet
3 . Implémenter un ordonnancement préemptif à priorités.  
Pour un quantum de 2 (cette valeur pourra être une donnée du problème), il produira, par exemple : 

```python
{'PA', 'PA', 'PC', 'PC', 'PA', 'PA', 'PB', 'PB', 
 'PC', 'PC', 'PA', 'PA', 'PB', 'PB', 'PC', 'PC', 
 'PA', 'PA', 'PA', 'PA', 'PC', 'PC'}
```

Une première version pourra considérer que tous les processus sont créés au temps $t=0$, c'est à dire ignorer les valeurs $t_0$.

#### Ordonnancement à priorités
Une priorité est, maintenant, attachée à chaque processus.  
Un processus est d'autant plus prioritaire que cette valeur est élevée.  

On aura, par exemple :

In [ ]:
data = [
    {'nom':'PA', 't0':0, 'duree':10, 'priorite':1},
    {'nom':'PB', 't0':3, 'duree':4,  'priorite':2},
    {'nom':'PC', 't0':2, 'duree':8,  'priorite':3},
]

4 . Implémenter un ordonnacement préemptif à priorités.  

Sur l'exemple, il produira :

```python
{'PA', 'PA', 
 'PC', 'PC', 'PC', 'PC', 'PC', 'PC', 'PC', 'PC',
 'PB', 'PB', 'PB', 'PB', 
 'PA', 'PA', 'PA', 'PA', 'PA', 'PA', 'PA', 'PA'}
```

### Exercice 8
Le programme [`mandelbrot.py`](Fichiers/mandelbrot.py) permet de produire une image fractale de Mandelbrot.  
La production de cette image est un calcul complexe qui occupe le microprocesseur pendant un certain temps.

Pour accélérer la production de l'image, l'idée est de confier le calcul de différentes fractions de l'image à différents microprocesseurs, puis d'assembler les fragments ainsi obtenus en une seule image.

<div style="text-align: center">
   <img alt="Mandelbrot" src="Images/mandelbrot.png" > 
</div>

Pour utiliser ce [programme](Fichiers/mandelbrot.py), nous pouvons simplement taper, dans un terminal, la commande suivante :

```
utilisateur@machine:~$ mandelbrot.py -2.0 -1.5 1.0 1.5 1920 1080 i.png

```
Cette commande produit une image Mandelbrot des abscisses $-2$ à $1$ et des ordonnées $-1,5$ à $1,5$.  
L'image de résolution *full HD 1920 x 1080* est produite dans le fichier `i.png`.

La production de cette image est un clacul complexe qui occupe le microprocesseur pendant un certain temps.  
Il est possible de mesurer le temps d'exécution du programme avec la commande [`time`](https://pubs.opengroup.org/onlinepubs/9699919799/utilities/time.html).

```
utilisateur@machine:~$ time python3 mandelbrot.py -2.0 -1.5 1.0 1.5 1920 1080 i.png

```

Le programme [`coller.py`](Fichiers/coller.py) assemble 4 images en mosaïque pour produire une image unique.  

Ce programme permet, par exemple, d'assembler 4 images `i1.png`, `i2.png`, `i3.png` et `i4.png` en une seule image `i.png` comme suit : 

```
utilisateur@machine:~$ python3 coller.py i1.png i2.png i3.png i4.png i.png

```
Pour accélérer la production des images Mandelbrot, l'idée est de confier le calcul d'une fraction de l'image à différents microprocesseurs, puis d'assembler les fragments ainsi obtenus en une seule image.

La fonction [`os.cpu_count`](https://docs.python.org/fr/3/library/os.html#os.cpu_count) renvoie le nombre de processeurs de l'ordinateur.

Il est possible de lancer plusieurs commandes, en tâche de fond, en les faisant suivre d'un `&`.  
Par ailleurs, il est possible d'attendre la fin de l'exécution des programmes précédemment lancés en utilisant la commande [`wait`](https://pubs.opengroup.org/onlinepubs/9699919799/utilities/wait.html).  
Ainsi `p1 & p2 & wait ; p3` lance les commandes `p1` et `p2` en tâche de fond, puis attend que ces commandes soieent terminées avant d'exécuter la commande `p3`.  
Les processus `p1` et `p2` s'exécutent, possiblement en parallèle, sur des microprocesseurs de l'ordinateur.

1 . Proposer une commande qui calcule, en parallèle, les 4 quartiers de la fractale de Mandelbrot sur le rectangle $[-2;1][-1,5;1,5]$ puis assemble les quartiers d'image obtenus dans une image unique `i.png`.

Pour répondre à la question suivante, il est possible de mesurer le temps d'exécution d'une commande composée en la mettant *entre parenthèses*.  
Par exemple, pour connaître le temps d'exécution de `p1 & p2 & wait ; p3`, il suffit de faire : 
```
utilisateur@machine:~$ time (p1 & p2 & wait ; p3)

```
2 . A l'aide de la commande `time`, déterminer le temps d'exécution nécessaire pour produire l'image de la fractale de Mandelbrot sur un seul processeur (tel que le porpose l'exemple d'utilisation de `mandelbrot.py` précédent) ainsi que le temps d'exécution de la solution à la question précédente.

La commande `time` produit, en fait, 3 informations temporelles.  
Elle indique le temps `real`, c'est-à-dire le temps observé par l'utilisateur, le temps `user` qui est le temps de processeur utilisé par le programme et le temps `sys` qui est le temps processeur utilisé par le système d'exploitation pour gérer les demandes du programme.

3 . Commenter les temps d'exécution observés.  
Il s'agit, en premier lieu, d'expliquer la différence de temps réel d'exécution observé, mais aussi les différences entre les temps d'exécution `user` et `sys`.

## Travaux pratiques
* [La cuisine du système](Travaux_Pratiques/TP_Cuisine_systeme.ipynb)

## Liens :
* Interstices : [Le ballet des processus dans un système d’exploitation](https://interstices.info/le-ballet-des-processus-dans-un-systeme-dexploitation/)
* Interstices : [À quoi sert un système d’exploitation ?](https://interstices.info/a-quoi-sert-un-systeme-dexploitation/)
* Société Informatique de France : [Brève histoire des systèmes d'exploitation](https://www.societe-informatique-de-france.fr/wp-content/uploads/2016/04/1024-no8-histoire-SE.pdf)
* [Kernel Shark](https://kernelshark.org/) est un logiciel permettant de visualiser l'exécution ainsi que la localisation et la temporalité de cer derniers.
    * Exemple de trace capturée : [trace.dat](Fichiers/trace.dat)
* Simulateur [BSim](https://computationstructures.org/exercises/sandboxes/bsim.html) pour une architecture 32 bits (développé par le MIT)  
Il permet de programmer en langage assembleur et observer l’exécution du programme qui s'exécute au sein d'un processeur 32 bits.
    * [Logiciel](https://computationstructures.org/exercises/sandboxes/bsim.html)
    * [Architecture](https://computationstructures.org/notes/pdfs/betadiagram.pdf)
    * [Documentation](https://computationstructures.org/notes/pdfs/beta.pdf)
    * [Instruction](https://computationstructures.org/notes/pdfs/betainst.pdf)
    * [OSTiny](https://jdolivet.github.io/NSI-Cours/Premi%C3%A8re/Sites/BSim/bsim/bsim_TinyOS%2B.html) : un mini système d'exploitation écrit sur le simulateur.  
    Il faut assembler et exécuter en avance rapide.  
    Il prend en charge le clavier et la souris dans la console.
* Architecture All Access - Modern CPU Architecture (Vidéo INTEL) : 
    * [Part 1 – Key Concepts](https://ladigitale.dev/digiplay/#/v/629940513491a)
    * [Part 2 – Microarchitecture Deep Dive](https://ladigitale.dev/digiplay/#/v/629940d6a261c)
* [The Deadlock Empire](https://deadlockempire.github.io/)